#### Libraries Import

In [1]:
import pandas as pd
import numpy as np
import altair as alt
import janitor
from datetime import datetime, timedelta
import calendar
from sklearn.model_selection import train_test_split
import seaborn as sn
import matplotlib.pyplot as plt
from altair_saver import save
alt.renderers.enable('default')
alt.data_transformers.enable('json') 

DataTransformerRegistry.enable('json')

#### File Reading

# Activations

In [2]:
act_2018 = pd.read_excel('../../../../data/V2 data 5.1.2020/2018.xlsx')
act_2019 = pd.read_excel('../../../../data/V2 data 5.1.2020/2019.xlsx')
act_2020 = pd.read_excel('../../../../data/V2 data 5.1.2020/2020.xlsx')

In [3]:
act_df = pd.concat([act_2018,act_2019,act_2020])

In [4]:
act_df.head(2)

,Total,Province,PostalCode,Banner,StoreName,Username,RoleName,ActType,DateCreatedAtMidnight
0,2,AB,T1B 2R4,TB,TB Medicine Hat Mall Kiosk 230 1051,H5Y,XEmployee,COP,2018-01-03
1,1,AB,T1B 2R4,TB,TB Medicine Hat Mall Kiosk 230 1051,H5Y,XEmployee,COP,2018-01-23


In [5]:
act_df["month"] =act_df['DateCreatedAtMidnight'].dt.month
act_df["year"] =act_df['DateCreatedAtMidnight'].dt.year
act_df["period"] = act_df.DateCreatedAtMidnight.dt.strftime("%d-%m-%Y").str[3:10]

In [6]:
act_df.head(2)

,Total,Province,PostalCode,Banner,StoreName,Username,RoleName,ActType,DateCreatedAtMidnight,month,year,period
0,2,AB,T1B 2R4,TB,TB Medicine Hat Mall Kiosk 230 1051,H5Y,XEmployee,COP,2018-01-03,1,2018,01-2018
1,1,AB,T1B 2R4,TB,TB Medicine Hat Mall Kiosk 230 1051,H5Y,XEmployee,COP,2018-01-23,1,2018,01-2018


## Activations per month

In [7]:
alt.Chart(act_df).mark_line(point = True).encode(
    alt.X('yearmonth(DateCreatedAtMidnight):O', title='Period'),
    alt.Y('sum(Total):Q', title='Sum Activations')
).properties(title = "Total Activations per Month")

alt.Chart(...)

Conclusions:
+ We should exclude 2020 from performance window
+ February Onwards to be excluded

#### Average activations per month (Excluding 2020)

In [8]:
act_df[act_df.DateCreatedAtMidnight < "2020-01-01"].groupby('period')['Total'].agg(['sum']).agg(['mean'])

,sum
mean,97658.5


#### Activations per month, per banner (total,average)

In [9]:
alt.Chart(act_df).mark_line(point = True).encode(
    alt.X('yearmonth(DateCreatedAtMidnight):O', title='Period'),
    alt.Y('sum(Total):Q', title='Sum Activations'),
    color = "Banner"
).properties(title = "Total Activations per Month per Banner")

alt.Chart(...)

#### Average activations per month per banner (Excluding 2020)

In [10]:
act_df[act_df.DateCreatedAtMidnight < "2020-01-01"].groupby(['Banner','period'])['Total'].agg(['sum']).groupby('Banner').agg(['mean'])

,sum
,mean
Banner,
Co,0.000000
TB,23864.875000
WE,38482.583333
WW,35311.041667


#### Activations per Employee per Month

In [11]:
act_df_no2020 = act_df[act_df.DateCreatedAtMidnight < "2020-01-01"]

In [12]:
avg_monthly_activation_per_employee = act_df_no2020.groupby(['Username','period'])['Total'].agg(['sum']).groupby('Username').agg(['mean'])
avg_monthly_activation_per_employee.columns = avg_monthly_activation_per_employee.columns.droplevel(0)

In [13]:
alt.Chart(avg_monthly_activation_per_employee).mark_area().encode(
    alt.X("mean:Q", bin=alt.Bin(extent=[0, 200], step=5),title ="Monthly Activations"),
    y='count()',
).properties(title = "Avg Montly Activations per Username")

alt.Chart(...)

### Activations per Storename

In [14]:
avg_monthly_activation_per_store = act_df_no2020.groupby(['StoreName','period'])['Total'].agg(['sum']).groupby('StoreName').agg(['mean'])
avg_monthly_activation_per_store.columns = avg_monthly_activation_per_store.columns.droplevel(0)
avg_monthly_activation_per_store = avg_monthly_activation_per_store.reset_index().sort_values(['mean'],ascending = False)

In [15]:
alt.Chart(avg_monthly_activation_per_store).mark_bar().encode(
    alt.X("mean:Q", bin=alt.Bin(extent=[0, 200], step=5),title ="Monthly Activations"),
    y='count()',
).properties(title = "Avg Montly Activations per Store")

alt.Chart(...)

In [16]:
alt.Chart(avg_monthly_activation_per_store).mark_bar().encode(
    alt.X("StoreName",title ="Store Name",sort = '-y'),
    y='mean:Q'
).properties(title = "Avg Montly Activations per Store")

alt.Chart(...)

### Activations per Province

In [17]:
avg_monthly_activation_per_province = act_df_no2020.groupby(['Province','period'])['Total'].agg(['sum']).groupby('Province').agg(['mean'])
avg_monthly_activation_per_province.columns = avg_monthly_activation_per_province.columns.droplevel(0)
avg_monthly_activation_per_province = avg_monthly_activation_per_province.reset_index()

In [18]:
avg_monthly_activation_per_province.head(2)

,Province,mean
0,AB,10779.625000
1,BC,14965.666667


In [19]:
alt.Chart(avg_monthly_activation_per_province).mark_bar().encode(
    alt.X("Province",title ="Province",sort = '-y'),
    y='mean:Q'
).properties(title = "Avg Montly Activations per Province")

alt.Chart(...)

#### Helper functions

In [20]:
def str_cols_to_date (df,col_list,frmt='%A, %B %d, %Y'):
    df[col_list] = df[col_list].applymap(lambda x:datetime.strptime(x,frmt))
    #print("columns updated in original dataframe")
    #print(df.head(5))
    return df

### Extracting Data from New Hires Table

In [21]:
new_hires = pd.read_excel('../../../../data/V2 data 5.1.2020/New Hires - 2020.04.28.xlsx').clean_names()
new_hires.head(5)

,employee_code,job_title_description,original_hire_date,recent_hire_date_if_applicable_,seniority_date_greater_than_hire_or_rehire_,position_status,termination_date,termination_reason_description,gender,terminated_,rehired_,report_date,current_tenure
0,B5H,SALES ASSOCIATE,"Thursday, May 24, 2007","Wednesday, July 31, 2019","Wednesday, July 31, 2019",Active,NaN,NaN,Male,Yes,Yes,"Wednesday, July 31, 2019",272
1,BDP,SALES MANAGER,"Sunday, August 26, 2007","Monday, July 29, 2019","Monday, July 29, 2019",Active,NaN,NaN,Male,Yes,Yes,"Wednesday, July 31, 2019",274
2,C5J,SALES ASSOCIATE,"Monday, July 7, 2008","Monday, January 8, 2018","Monday, January 8, 2018",Active,NaN,NaN,Male,Yes,Yes,"Wednesday, January 31, 2018",841
3,CKH,ASSISTANT MANAGER,"Wednesday, December 5, 2018",NaN,"Wednesday, December 5, 2018",Active,NaN,NaN,Male,Yes,No,"Monday, December 31, 2018",510
4,DJR,SALES ASSOCIATE,"Tuesday, April 24, 2018",NaN,"Tuesday, April 24, 2018",Active,NaN,NaN,Male,Yes,No,"Monday, April 30, 2018",735


In [22]:
new_hires_df_attr = new_hires[['employee_code','job_title_description','seniority_date_greater_than_hire_or_rehire_','rehired_']]
new_hires_df_attr.head()

,employee_code,job_title_description,seniority_date_greater_than_hire_or_rehire_,rehired_
0,B5H,SALES ASSOCIATE,"Wednesday, July 31, 2019",Yes
1,BDP,SALES MANAGER,"Monday, July 29, 2019",Yes
2,C5J,SALES ASSOCIATE,"Monday, January 8, 2018",Yes
3,CKH,ASSISTANT MANAGER,"Wednesday, December 5, 2018",No
4,DJR,SALES ASSOCIATE,"Tuesday, April 24, 2018",No


In [23]:
str_cols_to_date(new_hires_df_attr,['seniority_date_greater_than_hire_or_rehire_'])

C:\Users\NIW\Anaconda3\lib\site-packages\pandas\core\frame.py:3509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,employee_code,job_title_description,seniority_date_greater_than_hire_or_rehire_,rehired_
0,B5H,SALES ASSOCIATE,2019-07-31,Yes
1,BDP,SALES MANAGER,2019-07-29,Yes
2,C5J,SALES ASSOCIATE,2018-01-08,Yes
3,CKH,ASSISTANT MANAGER,2018-12-05,No
4,DJR,SALES ASSOCIATE,2018-04-24,No
...,...,...,...,...
1905,NIR,SALES ASSOCIATE,2020-03-19,No
1906,NIS,SALES ASSOCIATE,2020-03-16,No
1907,NIT,SALES ASSOCIATE,2020-03-17,No
1908,NIU,SALES ASSOCIATE,2020-03-18,No


In [24]:
#### extract snapshot of latest hire
new_hires_df_attr = new_hires_df_attr[new_hires_df_attr.groupby('employee_code').seniority_date_greater_than_hire_or_rehire_.transform('max')==new_hires_df_attr['seniority_date_greater_than_hire_or_rehire_']]

In [25]:
## Do we have duplicates?
print(new_hires_df_attr.size)
print(new_hires_df_attr.drop_duplicates().size)

7544
7540


In [26]:
##yes...lets explore:
new_hires_df_attr[new_hires_df_attr.duplicated()]

,employee_code,job_title_description,seniority_date_greater_than_hire_or_rehire_,rehired_
95,L35,SALES ASSOCIATE,2018-01-29,Yes


In [27]:
## How was this employee reported on the original new_hire dataset?
new_hires[new_hires.employee_code.isin(["L35","JUE"])]

,employee_code,job_title_description,original_hire_date,recent_hire_date_if_applicable_,seniority_date_greater_than_hire_or_rehire_,position_status,termination_date,termination_reason_description,gender,terminated_,rehired_,report_date,current_tenure
53,JUE,SALES ASSOCIATE,"Wednesday, July 11, 2018","Wednesday, July 11, 2018","Wednesday, July 11, 2018",Active,NaN,NaN,Male,Yes,Yes,"Tuesday, July 31, 2018",657
54,JUE,SALES ASSOCIATE,"Wednesday, July 11, 2018",NaN,"Wednesday, July 11, 2018",Terminated,"Saturday, April 30, 2016",Quit - New Job - Better Commissions,Male,No,No,"Tuesday, July 31, 2018",657
94,L35,SALES ASSOCIATE,"Wednesday, January 25, 2017","Monday, January 29, 2018","Monday, January 29, 2018",Active,NaN,NaN,Female,Yes,Yes,"Wednesday, January 31, 2018",820
95,L35,SALES ASSOCIATE,"Wednesday, January 25, 2017","Monday, January 29, 2018","Monday, January 29, 2018",Terminated,"Monday, August 7, 2017",Quit - New Job - Better Opportunity,Female,No,Yes,"Wednesday, January 31, 2018",820


In [28]:
new_hires_df_attr = new_hires_df_attr.drop_duplicates()

In [29]:
new_hires_df_attr.head(2)

,employee_code,job_title_description,seniority_date_greater_than_hire_or_rehire_,rehired_
0,B5H,SALES ASSOCIATE,2019-07-31,Yes
1,BDP,SALES MANAGER,2019-07-29,Yes


In [30]:
new_hires_df_attr.employee_code.value_counts(ascending=False)

JUE    2
N1C    1
MUS    1
NC9    1
NEW    1
      ..
M2B    1
NEQ    1
N46    1
NFO    1
MQR    1
Name: employee_code, Length: 1884, dtype: int64

In [31]:
## extracting re_hires
rehires_df = new_hires_df_attr[new_hires_df_attr.rehired_ =="Yes"][['employee_code','rehired_']]

In [32]:
rehires_df.employee_code.value_counts()

KJB    1
M5F    1
L6R    1
M55    1
L88    1
      ..
L0Z    1
G84    1
L46    1
KQQ    1
LAP    1
Name: employee_code, Length: 132, dtype: int64

### Extracting Data from Terminations Table

In [33]:
terminations_df = pd.read_excel('../../../../data/V2 data 5.1.2020/Terminations - 2020.04.28.xlsx').clean_names()

In [34]:
terminations_df.head(2)

,employee_code,ax,seniority_date_greater_of_hire_or_rehire_,termination_date,job_title,termination_reason,worker_category_description,gender,birth_year,report_date,tenure_at_termination,generation,termination_type,tenure_group_termination,qty
0,A12,1326,"Monday, March 6, 2006","Thursday, May 2, 2019",SALES MANAGER,Dismissal,Permanent Full Time,Male,1981,"Friday, May 31, 2019",4805,Gen X: 1965-1982,Dismissal,More than a Year,1
1,A3I,1216,"Monday, August 17, 1992","Friday, July 19, 2019",SALES ASSOCIATE,Quit - Personal,Permanent Full Time,Male,1967,"Wednesday, July 31, 2019",9832,Gen X: 1965-1982,Resignation,More than a Year,1


In [35]:
terminations_df_attr = str_cols_to_date(terminations_df,['termination_date'])[['employee_code','termination_date','termination_reason','termination_type']]

In [36]:
terminations_df_attr.head(2)

,employee_code,termination_date,termination_reason,termination_type
0,A12,2019-05-02,Dismissal,Dismissal
1,A3I,2019-07-19,Quit - Personal,Resignation


In [37]:
## Obtaing snapshot of latest termination in case of multiple
terminations_df_attr = terminations_df_attr[terminations_df_attr.groupby('employee_code').termination_date.transform('max')==terminations_df_attr['termination_date']]

In [38]:
terminations_df_attr.employee_code.value_counts().value_counts()

1    1864
Name: employee_code, dtype: int64

### Extracting data from "Active + Leave" Table

In [39]:
employee_status_weekly_snapshot = pd.read_excel('../../../../data/V2 data 5.1.2020/Active + Leave (Weekly) - 2020_04_28.xlsx').clean_names()

In [40]:
employee_status_weekly_snapshot.head(2)

,employee_code,original_hire_date,recent_hire_date_if_applicable_,gender,job_title,position_status,worker_category,birth_year,report_date_week_ending_,generation,tenure,tenure_group,qty
0,A12,"Monday, March 6, 2006","Monday, March 6, 2006",Male,SALES MANAGER,Active,Permanent Full Time,1981,"Friday, January 5, 2018",Gen X: 1965-1982,4323,10-15 Years,1
1,A12,"Monday, March 6, 2006","Monday, March 6, 2006",Male,SALES MANAGER,Active,Permanent Full Time,1981,"Friday, January 12, 2018",Gen X: 1965-1982,4330,10-15 Years,1


In [41]:
employee_status_weekly_snapshot = str_cols_to_date(employee_status_weekly_snapshot,['recent_hire_date_if_applicable_','report_date_week_ending_'])

In [42]:
employee_status_weekly_snapshot.head(2)

,employee_code,original_hire_date,recent_hire_date_if_applicable_,gender,job_title,position_status,worker_category,birth_year,report_date_week_ending_,generation,tenure,tenure_group,qty
0,A12,"Monday, March 6, 2006",2006-03-06,Male,SALES MANAGER,Active,Permanent Full Time,1981,2018-01-05,Gen X: 1965-1982,4323,10-15 Years,1
1,A12,"Monday, March 6, 2006",2006-03-06,Male,SALES MANAGER,Active,Permanent Full Time,1981,2018-01-12,Gen X: 1965-1982,4330,10-15 Years,1


In [43]:
### t1 extract the features we are interested in at the moment of hiring.
employee_status_attr_t1 = employee_status_weekly_snapshot[['employee_code','recent_hire_date_if_applicable_','gender','job_title','worker_category','birth_year']]

In [44]:
# We remove duplicates
employee_status_attr_t1 = employee_status_attr_t1.drop_duplicates()

In [45]:
## After dropping duplicates we can observe that we have certain employee codes repeating itself. It means that certain attributes changed over time (most certainly job_title and/or worker_category)
employee_status_attr_t1.employee_code.value_counts().value_counts()

1    2884
2     426
3     105
4      19
6       1
Name: employee_code, dtype: int64

In [46]:
## If multiple hires, we are just interested in information from the latest hire onward
employee_status_attr_t1 = employee_status_attr_t1[employee_status_attr_t1.groupby('employee_code').recent_hire_date_if_applicable_.transform('max')==employee_status_attr_t1['recent_hire_date_if_applicable_']]

In [47]:
## We investigate a bit further in which are there employee codes repeating itself and why.

In [48]:
employee_status_attr_t1.employee_code.value_counts()

L92    4
LWP    4
L6H    4
KZ0    4
KE7    4
      ..
JDS    1
L01    1
MDU    1
L6M    1
LLR    1
Name: employee_code, Length: 3435, dtype: int64

In [49]:
employee_status_attr_t1[employee_status_attr_t1.employee_code =='JZA']

,employee_code,recent_hire_date_if_applicable_,gender,job_title,worker_category,birth_year
68643,JZA,2015-08-25,Male,SALES ASSOCIATE,Permanent Full Time,1989
68645,JZA,2015-08-25,Male,SALES MANAGER,Permanent Full Time,1989
68660,JZA,2015-08-25,Male,ASSISTANT MANAGER,Permanent Full Time,1989
68687,JZA,2015-08-25,Male,TEMPORARY MANAGER,Permanent Full Time,1989


In [50]:
## We confirm that duplicates are because of a change in an atttribute over time

In [51]:
### We create a second table, in order to just analyze reports after most recent hire
employee_status_attr_t2 = pd.merge(employee_status_attr_t1[['employee_code','recent_hire_date_if_applicable_']],employee_status_weekly_snapshot,on="employee_code",how="inner")

In [52]:
employee_status_weekly_snapshot.head(2)

,employee_code,original_hire_date,recent_hire_date_if_applicable_,gender,job_title,position_status,worker_category,birth_year,report_date_week_ending_,generation,tenure,tenure_group,qty
0,A12,"Monday, March 6, 2006",2006-03-06,Male,SALES MANAGER,Active,Permanent Full Time,1981,2018-01-05,Gen X: 1965-1982,4323,10-15 Years,1
1,A12,"Monday, March 6, 2006",2006-03-06,Male,SALES MANAGER,Active,Permanent Full Time,1981,2018-01-12,Gen X: 1965-1982,4330,10-15 Years,1


In [53]:
employee_snapshot_valid_reports = employee_status_weekly_snapshot[employee_status_weekly_snapshot['report_date_week_ending_'] >= employee_status_weekly_snapshot.groupby('employee_code').recent_hire_date_if_applicable_.transform('max')]

In [54]:
employee_snapshot_valid_reports = employee_status_weekly_snapshot[employee_status_weekly_snapshot['report_date_week_ending_'] >= employee_status_weekly_snapshot.groupby('employee_code').recent_hire_date_if_applicable_.transform('max')]

In [55]:
employee_snapshot_valid_reports.head(2)

,employee_code,original_hire_date,recent_hire_date_if_applicable_,gender,job_title,position_status,worker_category,birth_year,report_date_week_ending_,generation,tenure,tenure_group,qty
0,A12,"Monday, March 6, 2006",2006-03-06,Male,SALES MANAGER,Active,Permanent Full Time,1981,2018-01-05,Gen X: 1965-1982,4323,10-15 Years,1
1,A12,"Monday, March 6, 2006",2006-03-06,Male,SALES MANAGER,Active,Permanent Full Time,1981,2018-01-12,Gen X: 1965-1982,4330,10-15 Years,1


In [56]:
##t3 contains first (oldest) status report available since most recent hire
employee_status_attr_t3 = employee_snapshot_valid_reports[employee_snapshot_valid_reports.groupby('employee_code').report_date_week_ending_.transform('min')==employee_snapshot_valid_reports['report_date_week_ending_']]

In [57]:
employee_status_attr_t3.head(2)

,employee_code,original_hire_date,recent_hire_date_if_applicable_,gender,job_title,position_status,worker_category,birth_year,report_date_week_ending_,generation,tenure,tenure_group,qty
0,A12,"Monday, March 6, 2006",2006-03-06,Male,SALES MANAGER,Active,Permanent Full Time,1981,2018-01-05,Gen X: 1965-1982,4323,10-15 Years,1
114,A1P,"Tuesday, August 31, 1999",1999-08-31,Male,SALES MANAGER,Active,Permanent Full Time,1974,2018-01-05,Gen X: 1965-1982,6702,15+ Years,1


In [58]:
employee_status_attr_t3 = employee_status_attr_t3.drop_duplicates()

In [59]:
employee_status_attr_t3 = employee_status_attr_t3[['employee_code','recent_hire_date_if_applicable_','gender','job_title','worker_category','birth_year','generation']]

In [60]:
employee_status_attr_t3.rename(columns={'recent_hire_date_if_applicable_':"max_hire_date"},inplace=True)

In [61]:
employee_status_attr_t3.head()

,employee_code,max_hire_date,gender,job_title,worker_category,birth_year,generation
0,A12,2006-03-06,Male,SALES MANAGER,Permanent Full Time,1981,Gen X: 1965-1982
114,A1P,1999-08-31,Male,SALES MANAGER,Permanent Full Time,1974,Gen X: 1965-1982
190,A2M,2005-07-17,Male,SALES MANAGER,Permanent Full Time,1981,Gen X: 1965-1982
311,A3I,1992-08-17,Male,SALES ASSOCIATE,Permanent Full Time,1967,Gen X: 1965-1982
391,A4C,2006-01-17,Female,SALES MANAGER,Permanent Full Time,1977,Gen X: 1965-1982


In [62]:
## We confirm no duplicate employees
employee_status_attr_t3.employee_code.value_counts().value_counts()

1    3435
Name: employee_code, dtype: int64

#### Employee with Resume

In [63]:
resume_employee = pd.read_csv('../../../../data/clean_data/05182020_cleaned_english_resumes_V1.0.csv',index_col =0).clean_names()

In [64]:
resume_employee.head(2)

,employee_name,employee_code,store,raw_resume,resume_text,resume_bline,language,file_type,clean_text
0,"Abbasi, Samiee Z",N42,WW 374,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"Samiee Zahid Abbasi 61 Muscat Crescent Ajax, ...","['', '', '', '', '', '', '', '', '', '', '', '...",English,docx,Zahid Muscat Crescent look build great work fa...
1,"Abdi, Melika",N3O,TB 146,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,Skills Experience MELIKA ABDI Carmelo Avenue ...,"['', '', '', '', '', '', '', '', '', '', '', '...",English,pdf,Skills Experience Carmelo Avenue Ph ability wo...


In [65]:
resume_employee['resume_found'] = 1
resume_employee.head(2)

,employee_name,employee_code,store,raw_resume,resume_text,resume_bline,language,file_type,clean_text,resume_found
0,"Abbasi, Samiee Z",N42,WW 374,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"Samiee Zahid Abbasi 61 Muscat Crescent Ajax, ...","['', '', '', '', '', '', '', '', '', '', '', '...",English,docx,Zahid Muscat Crescent look build great work fa...,1
1,"Abdi, Melika",N3O,TB 146,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,Skills Experience MELIKA ABDI Carmelo Avenue ...,"['', '', '', '', '', '', '', '', '', '', '', '...",English,pdf,Skills Experience Carmelo Avenue Ph ability wo...,1


### Waterfall - Final Sample Size

In [66]:
employee_status_attr_t3.head()

,employee_code,max_hire_date,gender,job_title,worker_category,birth_year,generation
0,A12,2006-03-06,Male,SALES MANAGER,Permanent Full Time,1981,Gen X: 1965-1982
114,A1P,1999-08-31,Male,SALES MANAGER,Permanent Full Time,1974,Gen X: 1965-1982
190,A2M,2005-07-17,Male,SALES MANAGER,Permanent Full Time,1981,Gen X: 1965-1982
311,A3I,1992-08-17,Male,SALES ASSOCIATE,Permanent Full Time,1967,Gen X: 1965-1982
391,A4C,2006-01-17,Female,SALES MANAGER,Permanent Full Time,1977,Gen X: 1965-1982


In [67]:
terminations_df_attr

,employee_code,termination_date,termination_reason,termination_type
0,A12,2019-05-02,Dismissal,Dismissal
1,A3I,2019-07-19,Quit - Personal,Resignation
2,A4C,2018-09-07,Quit - New Job - Better Opportunity,Resignation
3,A65,2019-05-30,Quit - Personal,Resignation
4,A80,2019-09-11,Dismissal - Misconduct - Theft,Dismissal
...,...,...,...,...
1889,NHK,2020-03-02,Quit - Job Abandonment - Attendance,Resignation
1890,NHX,2020-03-16,Dismissal - Lack of Professionalism,Dismissal
1891,NS2,2018-04-30,Quit - New Job - Better Opportunity,Resignation
1892,SB1,2019-07-05,Quit - Personal,Resignation


In [68]:
## We merge initial status report features with termination features

In [69]:
full_df = pd.merge(employee_status_attr_t3,terminations_df_attr,on='employee_code',how='left')

In [70]:
full_df['tenure_at_termination'] = full_df['termination_date'] - full_df['max_hire_date']

In [71]:
### Negative tenure at termination, indicates employee had previously worked for the company and was rehired. 
full_df[full_df.tenure_at_termination < timedelta(days=0)].head(5)

,employee_code,max_hire_date,gender,job_title,worker_category,birth_year,generation,termination_date,termination_reason,termination_type,tenure_at_termination
228,F6Q,2019-05-28,Male,SALES ASSOCIATE,Permanent Full Time,1990,Millenial: 1983-1999,2019-03-06,Quit - New Job - Better Opportunity,Resignation,-83 days
310,G64,2019-07-09,Male,SALES ASSOCIATE,Permanent Full Time,1981,Gen X: 1965-1982,2019-04-30,Quit - Personal,Resignation,-70 days
455,I3R,2019-10-28,Male,SALES ASSOCIATE,Permanent Full Time,1994,Millenial: 1983-1999,2019-01-08,Quit - Personal,Resignation,-293 days
484,IB7,2020-02-24,Female,SALES ASSOCIATE,Permanent Full Time,1994,Millenial: 1983-1999,2019-07-30,Quit - Personal,Resignation,-209 days
510,IJ4,2019-10-15,Male,SALES ASSOCIATE,Permanent Full Time,1980,Gen X: 1965-1982,2018-02-28,Quit - New Job - Better Opportunity,Resignation,-594 days


In [72]:
full_df['end_perf_window'] = datetime(2020, 2, 29)

In [73]:
full_df['tenure_at_end_perf_window'] = full_df['end_perf_window'] - full_df['max_hire_date']

In [74]:
full_df.head(2)

,employee_code,max_hire_date,gender,job_title,worker_category,birth_year,generation,termination_date,termination_reason,termination_type,tenure_at_termination,end_perf_window,tenure_at_end_perf_window
0,A12,2006-03-06,Male,SALES MANAGER,Permanent Full Time,1981,Gen X: 1965-1982,2019-05-02,Dismissal,Dismissal,4805 days,2020-02-29,5108 days
1,A1P,1999-08-31,Male,SALES MANAGER,Permanent Full Time,1974,Gen X: 1965-1982,NaT,NaN,NaN,NaT,2020-02-29,7487 days


### Adding resume_data

In [75]:
resume_employee.head(5)

,employee_name,employee_code,store,raw_resume,resume_text,resume_bline,language,file_type,clean_text,resume_found
0,"Abbasi, Samiee Z",N42,WW 374,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"Samiee Zahid Abbasi 61 Muscat Crescent Ajax, ...","['', '', '', '', '', '', '', '', '', '', '', '...",English,docx,Zahid Muscat Crescent look build great work fa...,1
1,"Abdi, Melika",N3O,TB 146,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,Skills Experience MELIKA ABDI Carmelo Avenue ...,"['', '', '', '', '', '', '', '', '', '', '', '...",English,pdf,Skills Experience Carmelo Avenue Ph ability wo...,1
2,"Abdulali, Kaizar",BDP,WW 252,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,KAIZAR ABDULALI kaizar83@gmail.com | 780 990-...,"['', '', '', '', '', '', '', '', '', '', '', '...",English,pdf,thank consideration role Project Manager Busin...,1
3,"Abrams, James",MZV,WW 851,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"Michael Abrams 5214 Gerrish Street, Apt 221 C...","['', '', '', '', '', '', '', '', '', '', '', '...",English,docx,Gerrish Apt entrepreneur new customer acquisit...,1
4,"Abro, Muhammad Wali",N62,WW 394,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"WALI ABRO 4 Montpelier Street, Brampton, ON |...","['', '', '', '', '', '', '', '', '', '', '', '...",English,pdf,Montpelier OVERVIEW Languages Fluent English S...,1


In [76]:
full_df = pd.merge(full_df,resume_employee[['employee_code','employee_name','language','resume_found']],on='employee_code',how='left')
full_df.head(2)

,employee_code,max_hire_date,gender,job_title,worker_category,birth_year,generation,termination_date,termination_reason,termination_type,tenure_at_termination,end_perf_window,tenure_at_end_perf_window,employee_name,language,resume_found
0,A12,2006-03-06,Male,SALES MANAGER,Permanent Full Time,1981,Gen X: 1965-1982,2019-05-02,Dismissal,Dismissal,4805 days,2020-02-29,5108 days,NaN,NaN,NaN
1,A1P,1999-08-31,Male,SALES MANAGER,Permanent Full Time,1974,Gen X: 1965-1982,NaT,NaN,NaN,NaT,2020-02-29,7487 days,NaN,NaN,NaN


In [77]:
full_df.head(2)

,employee_code,max_hire_date,gender,job_title,worker_category,birth_year,generation,termination_date,termination_reason,termination_type,tenure_at_termination,end_perf_window,tenure_at_end_perf_window,employee_name,language,resume_found
0,A12,2006-03-06,Male,SALES MANAGER,Permanent Full Time,1981,Gen X: 1965-1982,2019-05-02,Dismissal,Dismissal,4805 days,2020-02-29,5108 days,NaN,NaN,NaN
1,A1P,1999-08-31,Male,SALES MANAGER,Permanent Full Time,1974,Gen X: 1965-1982,NaT,NaN,NaN,NaT,2020-02-29,7487 days,NaN,NaN,NaN


### Adding Performance Data (employee_plateau)

In [78]:
## We start by creating a datetime indicanting the report time

In [79]:
import pandas as pd

In [80]:
perf_plateau_df = pd.read_excel('../../../../data/V2 data 5.1.2020/employee_plateau.xlsx').clean_names()

In [81]:
perf_plateau_df['perf_month'] = (perf_plateau_df['year'].astype(str) + '-' +
                                  perf_plateau_df['month'].astype(str).str[0:3])

In [82]:
perf_plateau_df.head(5)

,username,ctn_plateau,month,year,high_performer,perf_month
0,A7R,2,July,2019,Yes,2019-Jul
1,A7S,2,July,2019,Yes,2019-Jul
2,AE1,2,July,2019,Yes,2019-Jul
3,AGK,2,July,2019,Yes,2019-Jul
4,AJ7,2,July,2019,Yes,2019-Jul


In [83]:
str_cols_to_date(perf_plateau_df,['perf_month'],'%Y-%b')

,username,ctn_plateau,month,year,high_performer,perf_month
0,A7R,2,July,2019,Yes,2019-07-01
1,A7S,2,July,2019,Yes,2019-07-01
2,AE1,2,July,2019,Yes,2019-07-01
3,AGK,2,July,2019,Yes,2019-07-01
4,AJ7,2,July,2019,Yes,2019-07-01
...,...,...,...,...,...,...
23333,NHF,0,Feb,2020,No,2020-02-01
23334,NHH,0,Feb,2020,No,2020-02-01
23335,NHP,0,Feb,2020,No,2020-02-01
23336,NHS,0,Feb,2020,No,2020-02-01


In [84]:
## We then change the date to reflect the end of month date

In [85]:
perf_plateau_df['perf_month']=perf_plateau_df.perf_month.apply(lambda x: x.replace(day =calendar.monthrange(x.year,x.month)[1]))

In [86]:
perf_plateau_df.head(2)

,username,ctn_plateau,month,year,high_performer,perf_month
0,A7R,2,July,2019,Yes,2019-07-31
1,A7S,2,July,2019,Yes,2019-07-31


In [87]:
## Creating binary flag for reaching mininum compensation plateau

In [88]:
perf_plateau_df['high_perf_flag'] = perf_plateau_df.high_performer.map(lambda x: 1 if x =="Yes" else 0)

In [89]:
perf_plateau_df.rename(columns={'username':'employee_code'},inplace=True)

In [90]:
perf_plateau_df.head(3)

,employee_code,ctn_plateau,month,year,high_performer,perf_month,high_perf_flag
0,A7R,2,July,2019,Yes,2019-07-31,1
1,A7S,2,July,2019,Yes,2019-07-31,1
2,AE1,2,July,2019,Yes,2019-07-31,1


In [91]:
## Add max hire date, just to analyze performance from latest hire date onwards

In [92]:
perf_plateau_df = pd.merge(perf_plateau_df,full_df[['employee_code','max_hire_date']], on= 'employee_code',how = 'inner')

In [93]:
perf_plateau_df.head(5)

,employee_code,ctn_plateau,month,year,high_performer,perf_month,high_perf_flag,max_hire_date
0,A7R,2,July,2019,Yes,2019-07-31,1,2011-09-26
1,A7R,2,Jan,2019,Yes,2019-01-31,1,2011-09-26
2,A7R,2,Feb,2019,Yes,2019-02-28,1,2011-09-26
3,A7R,2,September,2019,No,2019-09-30,0,2011-09-26
4,A7R,2,March,2019,No,2019-03-31,0,2011-09-26


In [94]:
## Just including performance from the latest hired period. Filtering
perf_plateau_df = perf_plateau_df[perf_plateau_df.perf_month >perf_plateau_df.max_hire_date]

In [95]:
## Double checking if any duplicates
print(perf_plateau_df.size)
print(perf_plateau_df.drop_duplicates().size)

184792
184792


In [96]:
#Existance of duplicates...lets explore
perf_plateau_df[perf_plateau_df.duplicated()]

,employee_code,ctn_plateau,month,year,high_performer,perf_month,high_perf_flag,max_hire_date


In [97]:
perf_plateau_df[perf_plateau_df.employee_code=="MSY"]

,employee_code,ctn_plateau,month,year,high_performer,perf_month,high_perf_flag,max_hire_date
15660,MSY,2,October,2019,Yes,2019-10-31,1,2019-08-13
15661,MSY,2,August,2019,No,2019-08-31,0,2019-08-13
15662,MSY,2,Jan,2020,Yes,2020-01-31,1,2019-08-13
15665,MSY,4,December,2019,Yes,2019-12-31,1,2019-08-13
15666,MSY,3,September,2019,Yes,2019-09-30,1,2019-08-13
15667,MSY,3,November,2019,Yes,2019-11-30,1,2019-08-13
15668,MSY,3,Feb,2020,Yes,2020-02-29,1,2019-08-13


In [98]:
### Now duplicates understood, we are going to remove them
perf_plateau_df = perf_plateau_df.drop_duplicates()

In [99]:
### Logic for ignoring first two months

In [100]:
perf_plateau_df = perf_plateau_df.sort_values(by=["employee_code","perf_month"])

In [101]:
perf_plateau_df["MOB"] = perf_plateau_df.groupby(['employee_code']).cumcount()+1

In [102]:
### Ignoring first two months

In [103]:
perf_plateau_df = perf_plateau_df[perf_plateau_df.MOB >2]

In [104]:
## Creating counts for performance classification
perf_plateau_df = perf_plateau_df[['employee_code','high_perf_flag']].groupby('employee_code').agg({'high_perf_flag': ['sum','count']})

In [105]:
perf_plateau_df.columns = perf_plateau_df.columns.droplevel(0)

In [106]:
perf_plateau_df = perf_plateau_df.reset_index()

In [107]:
perf_plateau_df.rename(columns={'sum':'months_high_perf','count':'months_with_perf'},inplace=True)

In [108]:
perf_plateau_df.head(2)

,employee_code,months_high_perf,months_with_perf
0,A12,1,1
1,A1P,8,10


In [109]:
perf_plateau_df['hp_perc'] = perf_plateau_df['months_high_perf']/perf_plateau_df['months_with_perf']

In [110]:
perf_plateau_df['hp_class'] = perf_plateau_df['hp_perc'].map(lambda x: 1 if x>= 0.75 else 0)
perf_plateau_df['perf_found'] = 1

In [111]:
perf_plateau_df.head(2)

,employee_code,months_high_perf,months_with_perf,hp_perc,hp_class,perf_found
0,A12,1,1,1.0,1,1
1,A1P,8,10,0.8,1,1


### End preparing Performance Information

### Adding performance info to full_df

In [112]:
full_df = pd.merge(full_df,perf_plateau_df,on='employee_code',how='left')
full_df.head(2)

,employee_code,max_hire_date,gender,job_title,worker_category,birth_year,generation,termination_date,termination_reason,termination_type,...,end_perf_window,tenure_at_end_perf_window,employee_name,language,resume_found,months_high_perf,months_with_perf,hp_perc,hp_class,perf_found
0,A12,2006-03-06,Male,SALES MANAGER,Permanent Full Time,1981,Gen X: 1965-1982,2019-05-02,Dismissal,Dismissal,...,2020-02-29,5108 days,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0
1,A1P,1999-08-31,Male,SALES MANAGER,Permanent Full Time,1974,Gen X: 1965-1982,NaT,NaN,NaN,...,2020-02-29,7487 days,NaN,NaN,NaN,8.0,10.0,0.8,1.0,1.0


In [113]:
full_df.columns

Index(['employee_code', 'max_hire_date', 'gender', 'job_title',
       'worker_category', 'birth_year', 'generation', 'termination_date',
       'termination_reason', 'termination_type', 'tenure_at_termination',
       'end_perf_window', 'tenure_at_end_perf_window', 'employee_name',
       'language', 'resume_found', 'months_high_perf', 'months_with_perf',
       'hp_perc', 'hp_class', 'perf_found'],
      dtype='object')

### Adding re_hire flag to  full_df

In [114]:
full_df = pd.merge(full_df,rehires_df,on="employee_code",how ="left")

In [115]:
full_df

,employee_code,max_hire_date,gender,job_title,worker_category,birth_year,generation,termination_date,termination_reason,termination_type,...,tenure_at_end_perf_window,employee_name,language,resume_found,months_high_perf,months_with_perf,hp_perc,hp_class,perf_found,rehired_
0,A12,2006-03-06,Male,SALES MANAGER,Permanent Full Time,1981,Gen X: 1965-1982,2019-05-02,Dismissal,Dismissal,...,5108 days,NaN,NaN,NaN,1.0,1.0,1.000000,1.0,1.0,NaN
1,A1P,1999-08-31,Male,SALES MANAGER,Permanent Full Time,1974,Gen X: 1965-1982,NaT,NaN,NaN,...,7487 days,NaN,NaN,NaN,8.0,10.0,0.800000,1.0,1.0,NaN
2,A2M,2005-07-17,Male,SALES MANAGER,Permanent Full Time,1981,Gen X: 1965-1982,NaT,NaN,NaN,...,5340 days,NaN,NaN,NaN,7.0,12.0,0.583333,0.0,1.0,NaN
3,A3I,1992-08-17,Male,SALES ASSOCIATE,Permanent Full Time,1967,Gen X: 1965-1982,2019-07-19,Quit - Personal,Resignation,...,10057 days,NaN,NaN,NaN,0.0,5.0,0.000000,0.0,1.0,NaN
4,A4C,2006-01-17,Female,SALES MANAGER,Permanent Full Time,1977,Gen X: 1965-1982,2018-09-07,Quit - New Job - Better Opportunity,Resignation,...,5156 days,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3430,TD3,2005-09-08,Male,DISTRICT TEAM LEADER,Permanent Full Time,1985,Millenial: 1983-1999,NaT,NaN,NaN,...,5287 days,NaN,NaN,NaN,10.0,12.0,0.833333,1.0,1.0,NaN
3431,TM1,2000-11-08,Female,SALES MANAGER,Permanent Full Time,1970,Gen X: 1965-1982,NaT,NaN,NaN,...,7052 days,NaN,NaN,NaN,4.0,8.0,0.500000,0.0,1.0,NaN
3432,TP2,2003-04-29,Female,SALES MANAGER,Permanent Full Time,1969,Gen X: 1965-1982,NaT,NaN,NaN,...,6150 days,NaN,NaN,NaN,3.0,12.0,0.250000,0.0,1.0,NaN
3433,YK1,2018-07-13,Male,ASSISTANT MANAGER,Permanent Full Time,1986,Millenial: 1983-1999,2018-12-08,Quit - Personal,Resignation,...,596 days,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Adding referral flag to  full_df

In [116]:
referral_df = pd.read_excel('../../../../data/V2 data 5.1.2020/Referrals - 2020.04.28.xlsx').clean_names()
referral_df['referral_flag'] = 1

In [117]:
referral_df.head(2)

,employee_code,new_hire_gender,job_title,start_date,referral_flag
0,J5M,Male,Sales Associate,"Friday, January 12, 2018",1
1,K1T,Male,Sales Associate,"Friday, August 3, 2018",1


In [118]:
full_df = pd.merge(full_df,referral_df[['employee_code','referral_flag']],on="employee_code",how ="left")

## Preparing Exclusion motives field for waterfall

In [119]:
def exclusions(df):
    if (df['tenure_at_termination'] < timedelta(days=90)) and (df['tenure_at_termination'] >= timedelta(days=0)):
        return "01-Low Tenure - Termination <= 90 days"
    elif (df['tenure_at_end_perf_window'] < timedelta(days=90)) and (df['tenure_at_end_perf_window'] >= timedelta(days=0)):
        return "02-Low Tenure - Recent Hiring <= 90 days of perf"
    elif (df['resume_found']!= 1):
        return "03-No Resume"
    elif (df['language']!= "English"):
        return "04-Resume not in English"
    elif (df['perf_found']!= 1):
        return "05-No Perf Information"
    else:
        return "06-Pass"

In [120]:
def exclusions_2(df):
    if (df['resume_found']!= 1):
        return "01-No Resume"
    elif (df['tenure_at_end_perf_window'] < timedelta(days=90)) and (df['tenure_at_end_perf_window'] >= timedelta(days=0)):
        return "02-Low Tenure - Recent Hiring <= 90 days of perf"
    elif (df['tenure_at_termination'] < timedelta(days=90)) and (df['tenure_at_termination'] >= timedelta(days=0)):
        return "03-Low Tenure - Termination <= 90 days"
    elif (df['perf_found']!= 1):
        return "04-No Perf Information"
    else:
        return "05-Pass"

In [121]:
full_df['exclusion_code'] = full_df.apply(exclusions, axis=1)

In [122]:
full_df[['employee_code','exclusion_code']].groupby('exclusion_code').count()

,employee_code
exclusion_code,
01-Low Tenure - Termination <= 90 days,402
02-Low Tenure - Recent Hiring <= 90 days of perf,114
03-No Resume,2556
05-No Perf Information,3
06-Pass,360


In [123]:
full_df.head(2)

,employee_code,max_hire_date,gender,job_title,worker_category,birth_year,generation,termination_date,termination_reason,termination_type,...,language,resume_found,months_high_perf,months_with_perf,hp_perc,hp_class,perf_found,rehired_,referral_flag,exclusion_code
0,A12,2006-03-06,Male,SALES MANAGER,Permanent Full Time,1981,Gen X: 1965-1982,2019-05-02,Dismissal,Dismissal,...,NaN,NaN,1.0,1.0,1.0,1.0,1.0,NaN,NaN,03-No Resume
1,A1P,1999-08-31,Male,SALES MANAGER,Permanent Full Time,1974,Gen X: 1965-1982,NaT,NaN,NaN,...,NaN,NaN,8.0,10.0,0.8,1.0,1.0,NaN,NaN,03-No Resume


In [124]:
full_df[full_df['employee_code']=="MWM"]

,employee_code,max_hire_date,gender,job_title,worker_category,birth_year,generation,termination_date,termination_reason,termination_type,...,language,resume_found,months_high_perf,months_with_perf,hp_perc,hp_class,perf_found,rehired_,referral_flag,exclusion_code
2745,MWM,2019-04-15,Male,ASSISTANT MANAGER,Permanent Full Time,1988,Millenial: 1983-1999,NaT,NaN,NaN,...,English,1.0,5.0,9.0,0.555556,0.0,1.0,NaN,NaN,06-Pass


In [125]:
final_sample_full= full_df[full_df['exclusion_code']=='06-Pass']
final_sample_target = final_sample_full['hp_class']

In [126]:
final_sample_full

,employee_code,max_hire_date,gender,job_title,worker_category,birth_year,generation,termination_date,termination_reason,termination_type,...,language,resume_found,months_high_perf,months_with_perf,hp_perc,hp_class,perf_found,rehired_,referral_flag,exclusion_code
41,B5H,2019-07-31,Male,SALES ASSOCIATE,Permanent Full Time,1983,Millenial: 1983-1999,2019-12-31,Quit - Return to School,Resignation,...,English,1.0,0.0,3.0,0.000000,0.0,1.0,Yes,NaN,06-Pass
52,BDP,2019-07-29,Male,SALES MANAGER,Permanent Full Time,1983,Millenial: 1983-1999,NaT,NaN,NaN,...,English,1.0,2.0,6.0,0.333333,0.0,1.0,Yes,NaN,06-Pass
228,F6Q,2019-05-28,Male,SALES ASSOCIATE,Permanent Full Time,1990,Millenial: 1983-1999,2019-03-06,Quit - New Job - Better Opportunity,Resignation,...,English,1.0,7.0,8.0,0.875000,1.0,1.0,Yes,NaN,06-Pass
342,GKH,2019-07-22,Male,SALES ASSOCIATE,Permanent Full Time,1990,Millenial: 1983-1999,2020-03-14,Quit - Personal,Resignation,...,English,1.0,6.0,6.0,1.000000,1.0,1.0,Yes,NaN,06-Pass
544,IP5,2019-03-08,Male,SALES ASSOCIATE,Permanent Full Time,1994,Millenial: 1983-1999,NaT,NaN,NaN,...,English,1.0,8.0,9.0,0.888889,1.0,1.0,NaN,NaN,06-Pass
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3251,ND8,2019-11-26,Male,SALES ASSOCIATE,Permanent Full Time,1992,Millenial: 1983-1999,NaT,NaN,NaN,...,English,1.0,0.0,2.0,0.000000,0.0,1.0,NaN,1.0,06-Pass
3252,ND9,2019-11-20,Male,SALES ASSOCIATE,Permanent Full Time,1990,Millenial: 1983-1999,NaT,NaN,NaN,...,English,1.0,0.0,2.0,0.000000,0.0,1.0,NaN,NaN,06-Pass
3254,NDB,2019-11-20,Male,SALES ASSOCIATE,Permanent Full Time,1994,Millenial: 1983-1999,NaT,NaN,NaN,...,English,1.0,0.0,2.0,0.000000,0.0,1.0,NaN,NaN,06-Pass
3256,NDE,2019-11-18,Male,SALES ASSOCIATE,Permanent Full Time,1986,Millenial: 1983-1999,NaT,NaN,NaN,...,English,1.0,0.0,1.0,0.000000,0.0,1.0,NaN,NaN,06-Pass


In [127]:
final_sample_full[['employee_code','hp_class']].groupby('hp_class').count()

,employee_code
hp_class,
0.0,268
1.0,92


In [128]:
final_sample_full.shape[0]

360

In [129]:
pd.DataFrame(final_sample_full.columns)

,0
0,employee_code
1,max_hire_date
2,gender
3,job_title
4,worker_category
5,birth_year
6,generation
7,termination_date
8,termination_reason
9,termination_type


In [130]:
train_dataset = final_sample_full.sample(frac=0.80,random_state=1234)
test_dataset = final_sample_full[~final_sample_full.index.isin(list(train_dataset.index))]
print(train_dataset.shape[0])
print(test_dataset.shape[0])

288
72


In [131]:
train_dataset[train_dataset['employee_code']=='MWM']

,employee_code,max_hire_date,gender,job_title,worker_category,birth_year,generation,termination_date,termination_reason,termination_type,...,language,resume_found,months_high_perf,months_with_perf,hp_perc,hp_class,perf_found,rehired_,referral_flag,exclusion_code
2745,MWM,2019-04-15,Male,ASSISTANT MANAGER,Permanent Full Time,1988,Millenial: 1983-1999,NaT,NaN,NaN,...,English,1.0,5.0,9.0,0.555556,0.0,1.0,NaN,NaN,06-Pass


In [132]:
train_dataset.to_csv('../../../data/V2 data 5.1.2020/train_dataset.csv')
test_dataset.to_csv('../../../data/V2 data 5.1.2020/test_dataset.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../../../data/V2 data 5.1.2020/train_dataset.csv'

In [142]:
train_dataset.sample(n=120).to_csv('../../../data/V2 data 5.1.2020/manual_extraction_sample.csv')

### EDA with training Dataset

In [134]:
train_dataset.head()

,employee_code,max_hire_date,gender,job_title,worker_category,birth_year,generation,termination_date,termination_reason,termination_type,...,language,resume_found,months_high_perf,months_with_perf,hp_perc,hp_class,perf_found,rehired_,referral_flag,exclusion_code
2823,MZ9,2019-05-13,Male,SALES ASSOCIATE,Permanent Full Time,2000,Gen Z: 2000-Present,NaT,NaN,NaN,...,English,1.0,7.0,8.0,0.875000,1.0,1.0,NaN,NaN,06-Pass
2645,MS0,2019-03-06,Male,SALES ASSOCIATE,Permanent Full Time,1988,Millenial: 1983-1999,NaT,NaN,NaN,...,English,1.0,7.0,9.0,0.777778,1.0,1.0,NaN,NaN,06-Pass
1398,LCP,2019-09-07,Male,SALES ASSOCIATE,Permanent Part Time,1997,Millenial: 1983-1999,2020-03-02,Quit - Personal,Resignation,...,English,1.0,2.0,4.0,0.500000,0.0,1.0,Yes,NaN,06-Pass
2799,MYC,2019-05-13,Male,SALES ASSOCIATE,Permanent Full Time,1997,Millenial: 1983-1999,2019-08-19,Quit - Personal,Resignation,...,English,1.0,0.0,1.0,0.000000,0.0,1.0,NaN,NaN,06-Pass
732,JQB,2019-09-04,Male,SALES ASSOCIATE,Permanent Full Time,1996,Millenial: 1983-1999,2020-01-30,Dismissal - Attendance - Missed Shifts,Dismissal,...,English,1.0,0.0,3.0,0.000000,0.0,1.0,Yes,NaN,06-Pass


# EDA

# 1. Activations

In [136]:
read_train_data_set =pd.read_csv('../../../../data/clean_data/train_dataset.csv',index_col=0,parse_dates=['max_hire_date','termination_date','end_perf_window'],infer_datetime_format=1)
read_train_data_set.head(2)

,employee_code,original_hire_date,max_hire_date,gender,job_title,position_status,worker_category,birth_year,report_date_week_ending_,generation,...,language,resume_found,months_high_perf,months_with_perf,hp_perc,hp_class,perf_found,rehired_,referral_flag,exclusion_code
2825,MZE,"Saturday, May 18, 2019",2019-05-18,Female,SALES ASSOCIATE,Active,Permanent Full Time,1997,2019-05-24,Millenial: 1983-1999,...,English,1.0,0.0,3.0,0.0,0.0,1.0,NaN,1.0,06-Pass
2639,MRU,"Friday, February 22, 2019",2019-02-22,Male,SALES ASSOCIATE,Active,Permanent Full Time,1991,2019-02-22,Millenial: 1983-1999,...,English,1.0,0.0,3.0,0.0,0.0,1.0,NaN,NaN,06-Pass


In [137]:
read_train_data_set.dtypes

employee_code                        object
original_hire_date                   object
max_hire_date                datetime64[ns]
gender                               object
job_title                            object
position_status                      object
worker_category                      object
birth_year                            int64
report_date_week_ending_             object
generation                           object
tenure                                int64
tenure_group                         object
qty                                   int64
termination_date             datetime64[ns]
termination_reason                   object
termination_type                     object
tenure_at_termination                object
end_perf_window              datetime64[ns]
tenure_at_end_perf_window            object
employee_name                        object
language                             object
resume_found                        float64
months_high_perf                

In [138]:
read_train_data_set['class_tag'] = read_train_data_set['hp_class'].apply(lambda x: "high-perf" if x==1 else "non-high-perf")

In [139]:
act_df.head(2)

,Total,Province,PostalCode,Banner,StoreName,Username,RoleName,ActType,DateCreatedAtMidnight,month,year,period
0,2,AB,T1B 2R4,TB,TB Medicine Hat Mall Kiosk 230 1051,H5Y,XEmployee,COP,2018-01-03,1,2018,01-2018
1,1,AB,T1B 2R4,TB,TB Medicine Hat Mall Kiosk 230 1051,H5Y,XEmployee,COP,2018-01-23,1,2018,01-2018


In [140]:
act_df.rename(columns={"Username":"employee_code"},inplace = True)

In [141]:
act_eda_df = pd.merge(read_train_data_set[['employee_code','max_hire_date','end_perf_window','class_tag','hp_class','hp_perc','rehired_','referral_flag','generation','gender']],act_df, on="employee_code",how='left')
act_eda_df.head(2)

,employee_code,max_hire_date,end_perf_window,class_tag,hp_class,hp_perc,rehired_,referral_flag,generation,gender,...,Province,PostalCode,Banner,StoreName,RoleName,ActType,DateCreatedAtMidnight,month,year,period
0,MZE,2019-05-18,2020-02-29,non-high-perf,0.0,0.0,NaN,1.0,Millenial: 1983-1999,Female,...,ON,M1P 4P5,WW,WW Scarborough Town Centre Inline 319 1823,XEmployee,COP,2019-05-21,5.0,2019.0,05-2019
1,MZE,2019-05-18,2020-02-29,non-high-perf,0.0,0.0,NaN,1.0,Millenial: 1983-1999,Female,...,ON,M1P 4P5,WW,WW Scarborough Town Centre Inline 319 1823,XEmployee,COP,2019-05-27,5.0,2019.0,05-2019


In [142]:
#only including activations from most recent hire onwards and within performance windows
act_eda_df = act_eda_df[(act_eda_df.DateCreatedAtMidnight>=act_eda_df.max_hire_date) & (act_eda_df.DateCreatedAtMidnight<=act_eda_df.end_perf_window)]

In [143]:
act_eda_monthly_activation_per_employee = act_eda_df.groupby(['employee_code','class_tag','period'])['Total'].agg(['sum']).groupby(['employee_code','class_tag']).agg(['mean'])
act_eda_monthly_activation_per_employee.columns = act_eda_monthly_activation_per_employee.columns.droplevel(0)
act_eda_monthly_activation_per_employee= act_eda_monthly_activation_per_employee.reset_index()
act_eda_monthly_activation_per_employee.head(2)

,employee_code,class_tag,mean
0,B5H,non-high-perf,20.333333
1,BDP,non-high-perf,28.375000


In [144]:
a_00 = alt.Chart(act_eda_monthly_activation_per_employee).mark_area().encode(
    alt.X("mean:Q", bin=alt.Bin(extent=[0, 200], step=5),title ="Monthly Activations"),
    y='count()'
).properties(title = "Avg Montly Activations per Employee")

In [145]:
a_00

alt.Chart(...)

In [146]:
a_01 = alt.Chart(act_eda_monthly_activation_per_employee).mark_area(opacity=0.3).encode(
    alt.X("mean:Q", bin=alt.Bin(extent=[0, 200], step=5),title ="Monthly Activations"),
    y=alt.Y('count()',stack=None),
    color = 'class_tag'
).properties(title = "Avg Montly Activations per Employee")

In [147]:
a_01

alt.Chart(...)

In [150]:
act_eda_df.head(2)

,employee_code,max_hire_date,end_perf_window,class_tag,hp_class,hp_perc,rehired_,referral_flag,generation,gender,...,Province,PostalCode,Banner,StoreName,RoleName,ActType,DateCreatedAtMidnight,month,year,period
0,MZE,2019-05-18,2020-02-29,non-high-perf,0.0,0.0,NaN,1.0,Millenial: 1983-1999,Female,...,ON,M1P 4P5,WW,WW Scarborough Town Centre Inline 319 1823,XEmployee,COP,2019-05-21,5.0,2019.0,05-2019
1,MZE,2019-05-18,2020-02-29,non-high-perf,0.0,0.0,NaN,1.0,Millenial: 1983-1999,Female,...,ON,M1P 4P5,WW,WW Scarborough Town Centre Inline 319 1823,XEmployee,COP,2019-05-27,5.0,2019.0,05-2019


In [151]:
act_eda_monthly_activation_banner = act_eda_df.groupby(['Banner','period'])['Total'].agg(['sum']).groupby(['Banner']).agg(['mean'])
act_eda_monthly_activation_banner.columns = act_eda_monthly_activation_banner.columns.droplevel(0)
act_eda_monthly_activation_banner= act_eda_monthly_activation_banner.reset_index()
act_eda_monthly_activation_banner.head(2)

,Banner,mean
0,TB,1514.866667
1,WE,3489.357143


In [152]:
a_03 = alt.Chart(act_eda_monthly_activation_banner).mark_bar().encode(
    alt.X("Banner",title ="Banner",sort = '-y'),
    y='mean:Q'
).properties(title = "Avg Montly Activations per Banner")

In [153]:
a_03

alt.Chart(...)

In [154]:
act_eda_monthly_activation_province = act_eda_df.groupby(['Province','period'])['Total'].agg(['sum']).groupby(['Province']).agg(['mean'])
act_eda_monthly_activation_province.columns = act_eda_monthly_activation_province.columns.droplevel(0)
act_eda_monthly_activation_province= act_eda_monthly_activation_province.reset_index()
act_eda_monthly_activation_province.head(2)

,Province,mean
0,AB,1159.214286
1,BC,1200.857143


In [155]:
a_04 = alt.Chart(act_eda_monthly_activation_province).mark_bar().encode(
    alt.X("Province",title ="Province",sort = '-y'),
    y='mean:Q'
).properties(title = "Avg Montly Activations per Province")

In [156]:
a_04

alt.Chart(...)

# Performance Rate

In [157]:
a_05 = alt.Chart(read_train_data_set[['employee_code','hp_perc']]).mark_area(opacity=0.3).encode(
    alt.X("hp_perc:Q",title ="high_perf perc"),
    y=alt.Y('count()')
).properties(title = "High-perf rate Distribution")

In [158]:
a_05

alt.Chart(...)

In [159]:
a_06 = alt.Chart(read_train_data_set[['employee_code','hp_perc']]).transform_density(
    'hp_perc',
    as_=['hp_perc', 'density'],cumulative=True,
).mark_area().encode(
    x="hp_perc:Q",
    y='density:Q',
).properties(title = "Accuumalative Density")

In [160]:
a_06 

alt.Chart(...)

In [161]:
read_train_data_set[read_train_data_set.hp_perc >1][['employee_code','hp_perc','max_hire_date']].sort_values(by="hp_perc",ascending = False).head()

,employee_code,hp_perc,max_hire_date


In [162]:
perf_plateau_df[perf_plateau_df.employee_code == "MOC"]

,employee_code,months_high_perf,months_with_perf,hp_perc,hp_class,perf_found
1555,MOC,2,2,1.0,1,1


In [160]:
org_perf_data = pd.read_excel('../../../data/V2 data 5.1.2020/employee_plateau.xlsx').clean_names()

In [161]:
org_perf_data[org_perf_data.username=="MOC"]

,username,ctn_plateau,month,year,high_performer
5351,MOC,2,March,2019,No
6266,MOC,2,Feb,2020,Yes
6828,MOC,2,Jan,2020,Yes
8814,MOC,1,April,2019,No
11310,MOC,1,Feb,2019,No
11916,MOC,1,Jan,2019,No
17005,MOC,4,December,2019,Yes
18460,MOC,3,November,2019,Yes


# Employee Location

In [162]:
read_train_data_set.head(2)

,employee_code,max_hire_date,gender,job_title,worker_category,birth_year,generation,termination_date,termination_reason,termination_type,...,resume_found,months_high_perf,months_with_perf,hp_perc,hp_class,perf_found,rehired_,referral_flag,exclusion_code,class_tag
2823,MZ9,2019-05-13,Male,SALES ASSOCIATE,Permanent Full Time,2000,Gen Z: 2000-Present,NaT,NaN,NaN,...,1.0,7.0,8.0,0.875000,1.0,1.0,NaN,NaN,06-Pass,high-perf
2645,MS0,2019-03-06,Male,SALES ASSOCIATE,Permanent Full Time,1988,Millenial: 1983-1999,NaT,NaN,NaN,...,1.0,7.0,9.0,0.777778,1.0,1.0,NaN,NaN,06-Pass,high-perf


In [163]:
act_eda_df.head(2)

,employee_code,max_hire_date,end_perf_window,class_tag,hp_class,hp_perc,rehired_,referral_flag,generation,gender,...,Province,PostalCode,Banner,StoreName,RoleName,ActType,DateCreatedAtMidnight,month,year,period
0,MZ9,2019-05-13,2020-02-29,high-perf,1.0,0.875,NaN,NaN,Gen Z: 2000-Present,Male,...,AB,T5T 4J2,TB,TB West Edmonton Mall Kiosk 260 1302,Store Representative,NAC,2019-09-22,9.0,2019.0,09-2019
1,MZ9,2019-05-13,2020-02-29,high-perf,1.0,0.875,NaN,NaN,Gen Z: 2000-Present,Male,...,AB,T5T 4J2,TB,TB West Edmonton Mall Kiosk 260 1302,Store Representative,HUP,2019-10-03,10.0,2019.0,10-2019


In [164]:
employee_loc = act_eda_df[act_eda_df.groupby('employee_code').DateCreatedAtMidnight.transform('min')==act_eda_df['DateCreatedAtMidnight']]

In [165]:
employee_loc.columns

Index(['employee_code', 'max_hire_date', 'end_perf_window', 'class_tag',
       'hp_class', 'hp_perc', 'rehired_', 'referral_flag', 'generation',
       'gender', 'Total', 'Province', 'PostalCode', 'Banner', 'StoreName',
       'RoleName', 'ActType', 'DateCreatedAtMidnight', 'month', 'year',
       'period'],
      dtype='object')

In [166]:
employee_loc = employee_loc[['employee_code','hp_class','rehired_','referral_flag','gender','Province','Banner','StoreName','RoleName']]

In [167]:
employee_loc = employee_loc.drop_duplicates()

In [168]:
employee_loc.head(2)

,employee_code,hp_class,rehired_,referral_flag,gender,Province,Banner,StoreName,RoleName
244,MZ9,1.0,NaN,NaN,Male,AB,WW,WW West Edmonton Mall Upper Kiosk 253 1075,Store Representative
668,MS0,1.0,NaN,NaN,Male,ON,WE,WE Etobicoke 524 1214,Store Representative


In [169]:
a_07 = alt.Chart(employee_loc).mark_bar().encode(
    alt.X('Banner:O', title='Banner'),
    alt.Y('count()', title='High Performer Rate'),
    color = "Banner"
).properties(title = "Employee Distribution per Banner (Initial Location)")

b_07 = alt.Chart(employee_loc).mark_bar().encode(
    alt.X('Banner:O', title='Banner'),
    alt.Y('mean(hp_class):Q', title='High Performer Rate'),
    color = 'mean(hp_class):Q'
).properties(title = "High Performance Rate")

In [170]:
a_07|b_07

alt.HConcatChart(...)

If the metric is doing a fair comparison, is the hiring process differente for WE? Are the best profiles hired for WE?

In [171]:
a_08 = alt.Chart(employee_loc).mark_bar().encode(
    alt.X('Province:O', title='Province'),
    alt.Y('count()', title='High Performer Rate'),
    color = "Province"
).properties(title = "Employee Distribution per Province (Initial Location)")

b_08 = alt.Chart(employee_loc).mark_bar().encode(
    alt.X('Province:O', title='Province'),
    alt.Y('mean(hp_class):Q', title='High Performer Rate'),
    color = "Province"
).properties(title = "High Performance Rate")

In [172]:
a_08|b_08

alt.HConcatChart(...)

In [173]:
a_09 = alt.Chart(employee_loc).mark_bar().encode(
    alt.X('StoreName:O', title='StoreName',sort = '-y'),
    y = alt.Y('count()', title='High Performer Rate'),
    color = "StoreName"
).properties(title = "Employee Distribution per Store (Initial Location)")

b_09 = alt.Chart(employee_loc).mark_bar().encode(
    x = alt.X('StoreName:O', title='StoreName',sort = '-y'),
    y = alt.Y('mean(hp_class):Q', title='High Performer Rate'),
    color = "StoreName"
).properties(title = "High Performance Rate")

In [174]:
a_09

alt.Chart(...)

In [175]:
perf_rate_per_store = employee_loc.groupby(['StoreName'])[['hp_class']].agg('mean')
perf_rate_per_store = perf_rate_per_store.reset_index()
perf_rate_per_store.head(2)

,StoreName,hp_class
0,TB Bayshore Shopping Centre Kiosk 421 1169,0.0
1,TB Bower Place 233 1053,0.0


In [176]:
a_10 = alt.Chart(perf_rate_per_store).mark_bar(opacity=0.3).encode(
    alt.X("hp_class:Q",title ="Avg Perf Rate per Store"),
    y=alt.Y('count()')
).properties(title = "Avg Perf Rate per Store")

In [177]:
a_10

alt.Chart(...)

### Tabular Data Distribution

In [178]:
def employee_type(df):
    if (df['referral_flag'] ==1.0):
        return "referral"
    elif (df['rehired_'] =="Yes"):
        return "re-hire"
    else:
        return "new-hire"

In [179]:
read_train_data_set.rehired_.value_counts()

Yes    26
Name: rehired_, dtype: int64

In [180]:
read_train_data_set.head(2)

,employee_code,max_hire_date,gender,job_title,worker_category,birth_year,generation,termination_date,termination_reason,termination_type,...,resume_found,months_high_perf,months_with_perf,hp_perc,hp_class,perf_found,rehired_,referral_flag,exclusion_code,class_tag
2823,MZ9,2019-05-13,Male,SALES ASSOCIATE,Permanent Full Time,2000,Gen Z: 2000-Present,NaT,NaN,NaN,...,1.0,7.0,8.0,0.875000,1.0,1.0,NaN,NaN,06-Pass,high-perf
2645,MS0,2019-03-06,Male,SALES ASSOCIATE,Permanent Full Time,1988,Millenial: 1983-1999,NaT,NaN,NaN,...,1.0,7.0,9.0,0.777778,1.0,1.0,NaN,NaN,06-Pass,high-perf


In [181]:
read_train_data_set['employee_type'] = read_train_data_set.apply(employee_type, axis=1)

In [182]:
read_train_data_set.employee_type.value_counts()

new-hire    214
referral     49
re-hire      25
Name: employee_type, dtype: int64

In [183]:
a_11 = alt.Chart(read_train_data_set).mark_bar().encode(
    alt.X('employee_type:O', title='Employee Type'),
    alt.Y('count()', title='count'),
    color = "employee_type"
).properties(title = "Employee Type Distribution")

b_11 = alt.Chart(read_train_data_set).mark_bar().encode(
    alt.X('employee_type:O', title='Employee Type'),
    alt.Y('mean(hp_class):Q', title='High Performer Rate'),
    color = "employee_type"
).properties(title = "High Performance Rate per Employee Type")

In [184]:
a_11|b_11

alt.HConcatChart(...)

In [185]:
a_12 = alt.Chart(read_train_data_set).mark_bar().encode(
    alt.X('gender:O', title='Gender'),
    alt.Y('count()', title='count'),
    color = "gender"
).properties(title = "Gender Distribution")

b_12 = alt.Chart(read_train_data_set).mark_bar().encode(
    alt.X('gender:O', title='Gender'),
    alt.Y('mean(hp_class):Q', title='High Performer Rate'),
    color = "gender"
).properties(title = "High Performance Rate per Gender")

In [186]:
a_12|b_12

alt.HConcatChart(...)

In [187]:
read_train_data_set.dtypes

employee_code                        object
max_hire_date                datetime64[ns]
gender                               object
job_title                            object
worker_category                      object
birth_year                            int64
generation                           object
termination_date             datetime64[ns]
termination_reason                   object
termination_type                     object
tenure_at_termination                object
end_perf_window              datetime64[ns]
tenure_at_end_perf_window            object
language                             object
resume_found                        float64
months_high_perf                    float64
months_with_perf                    float64
hp_perc                             float64
hp_class                            float64
perf_found                          float64
rehired_                             object
referral_flag                       float64
exclusion_code                  

In [188]:
read_train_data_set["year_hired"] = pd.DatetimeIndex(read_train_data_set['max_hire_date']).year

In [189]:
read_train_data_set['age'] = read_train_data_set['year_hired'] -read_train_data_set['birth_year']

In [190]:
alt.Chart(read_train_data_set).mark_area().encode(
    alt.X("age:Q", bin=alt.Bin(extent=[15, 60], step=5),title ="Age"),
    y='count()'
).properties(title = "Employee Age Distribution")

alt.Chart(...)

In [191]:
a_13 = alt.Chart(read_train_data_set).mark_area(opacity=0.3).encode(
    alt.X("age:Q", bin=alt.Bin(extent=[0, 200], step=5),title ="Age"),
    y=alt.Y('count()',stack=None),
    color = 'class_tag'
).properties(title = "Avg Montly Activations per Employee")

In [192]:
a_13

alt.Chart(...)

In [193]:
a_14 = alt.Chart(read_train_data_set).mark_bar().encode(
    alt.X("age:Q", bin=alt.Bin(extent=[15, 60], step=5),title ="Age"),
    y=alt.Y('mean(hp_class):Q',stack=None),
    color = 'mean(hp_class):Q'
).properties(title = "Perf Rate per Age")

In [194]:
a_14

alt.Chart(...)

In [2]:
### Correlation Matrix

In [3]:
feats_df = pd.read_csv('C:/Users/NIW/Documents/Capstone Project/data/clean_data/manual_clean_training_dataset_v8.csv')

In [4]:
feats_df.corr()

,referral_flag,job_hopper,competitor_experience,Freedom_competitor_exp,Koodo_competitor_exp,Shaw_competitor_exp,Telus_competitor_exp,Bell_competitor_exp,Rogers_competitor_exp,The_Mobile_Shop_competitor_exp,...,no_jobs,hp_class,Clothing_and_Footwear_industry_exp,Consumer_electronics_industry_exp,Food_Service_industry_exp,Food-Convenience-Pharmacy_industry_exp,Other_industry_exp,Sport_Travel_Enterntain_Hotel_industry_exp,Telecommunications_industry_exp,unknown_industry_exp
referral_flag,1.000000,-0.048309,0.057233,-0.031039,0.022669,0.070738,0.077619,0.072055,0.056359,0.038192,...,-0.067175,-0.008394,-0.032810,-0.078410,0.094198,-0.002140,-0.000194,-0.070579,-0.003110,-0.080810
job_hopper,-0.048309,1.000000,-0.004291,0.093984,0.006373,0.054961,-0.073095,0.157604,-0.072659,-0.031033,...,0.110986,-0.069097,-0.021662,0.016207,0.047802,-0.062931,0.047194,0.046201,0.016042,0.085700
competitor_experience,0.057233,-0.004291,1.000000,0.177128,0.117041,0.082471,0.131087,0.283621,0.330002,0.247006,...,0.094229,0.186430,-0.114978,0.082053,-0.109094,-0.102772,-0.075168,-0.102419,0.766142,-0.101043
Freedom_competitor_exp,-0.031039,0.093984,0.177128,1.000000,-0.021315,-0.015019,-0.023873,0.023478,-0.060099,-0.044984,...,0.028616,-0.008724,-0.016063,-0.055375,-0.023580,-0.089218,-0.021712,-0.020801,0.102340,0.041119
Koodo_competitor_exp,0.022669,0.006373,0.117041,-0.021315,1.000000,-0.009924,-0.015775,-0.034130,-0.039712,-0.029724,...,-0.019442,-0.067250,-0.048352,-0.036590,-0.008249,0.015514,0.086926,0.032071,0.094672,-0.042946
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Food-Convenience-Pharmacy_industry_exp,-0.002140,-0.062931,-0.102772,-0.089218,0.015514,-0.041540,0.000695,-0.044429,-0.079339,-0.050459,...,0.112940,-0.037672,-0.052739,0.032565,-0.019989,1.000000,-0.113595,0.031288,-0.122156,0.051939
Other_industry_exp,-0.000194,0.047194,-0.075168,-0.021712,0.086926,0.018435,-0.052365,-0.016918,-0.008944,0.040127,...,0.210758,-0.107188,-0.083172,-0.146720,0.003460,-0.113595,1.000000,0.034316,-0.175856,0.049524
Sport_Travel_Enterntain_Hotel_industry_exp,-0.070579,0.046201,-0.102419,-0.020801,0.032071,-0.035511,-0.056445,-0.013121,-0.013809,-0.065407,...,0.236071,-0.105628,0.020332,-0.130925,0.099279,0.031288,0.034316,1.000000,-0.116584,-0.030506
Telecommunications_industry_exp,-0.003110,0.016042,0.766142,0.102340,0.094672,0.066709,0.106034,0.202601,0.266933,0.199799,...,0.052426,0.186350,-0.103052,0.018270,-0.102970,-0.122156,-0.175856,-0.116584,1.000000,-0.150649


In [29]:
feats_not_include_corr = ['background_highest_degree','rehired_','telco_electro_perc_group','clean_text','country_highest_degree','employee_code','highest_degree','no_job_categorical','read_score_categorical','recency_type_telco_electro_exp']

In [23]:
feats_df.drop(feats_not_include_corr, axis=1)

,rehired_,referral_flag,job_hopper,competitor_experience,Freedom_competitor_exp,Koodo_competitor_exp,Shaw_competitor_exp,Telus_competitor_exp,Bell_competitor_exp,Rogers_competitor_exp,...,no_jobs,hp_class,Clothing_and_Footwear_industry_exp,Consumer_electronics_industry_exp,Food_Service_industry_exp,Food-Convenience-Pharmacy_industry_exp,Other_industry_exp,Sport_Travel_Enterntain_Hotel_industry_exp,Telecommunications_industry_exp,unknown_industry_exp
0,No,1.0,0.0,1,0,0,0,0,0,0,...,2,0.0,0,0,0,0,0,0,1,1
1,No,0.0,0.0,1,0,0,0,0,0,0,...,3,0.0,0,0,0,0,1,0,1,1
2,Yes,0.0,0.0,0,0,0,0,0,0,0,...,6,0.0,0,0,0,0,1,0,1,1
3,No,0.0,0.0,0,0,0,0,0,0,0,...,6,1.0,1,0,0,0,0,0,0,1
4,Yes,0.0,0.0,0,0,0,0,0,0,0,...,3,0.0,0,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,No,0.0,0.0,1,0,0,0,0,0,0,...,4,0.0,0,1,0,0,0,0,1,0
284,No,0.0,0.0,0,0,0,0,0,0,0,...,3,0.0,0,0,1,0,0,0,1,0
285,No,0.0,0.0,1,0,0,0,0,0,0,...,3,0.0,1,0,1,0,0,0,1,0
286,No,0.0,1.0,0,0,0,0,0,0,0,...,5,0.0,0,0,1,0,1,0,0,1


In [62]:
df = feats_df.drop(feats_not_include_corr, axis=1)

def get_redundant_pairs(df):
    '''Get diagonal and lower triangular pairs of correlation matrix'''
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_top_abs_correlations(df, n=5):
    au_corr = df.corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    return au_corr[0:n]

In [48]:
top_corr_feats = get_top_abs_correlations(df, 30)
top_corr_feats = top_corr_feats.to_frame().reset_index()
top_corr_feats.rename(columns={"level_0":"Feature_1","level_1":"Feature_2",0:"corr_coef"},inplace="True")
top_corr_feats[top_corr_feats['corr_coef']>=0.50]

,Feature_1,Feature_2,corr_coef
0,clean_Flesch-Kincaid_readability,clean_automate_readability,0.999870
1,raw_Flesch-Kincaid_readability,raw_automate_readability,0.999517
2,clean_Flesch-Kincaid_readability,clean_Gunning_FOG_readability,0.999220
3,clean_Gunning_FOG_readability,clean_automate_readability,0.998955
4,raw_Flesch-Kincaid_readability,raw_Gunning_FOG_readability,0.998951
5,raw_Gunning_FOG_readability,raw_automate_readability,0.997782
6,clean_Gunning_FOG_readability,clean_dale_chall_readability,0.989410
7,clean_Flesch-Kincaid_readability,clean_dale_chall_readability,0.988567
8,clean_automate_readability,clean_dale_chall_readability,0.987660
9,raw_dale_chall_readability,raw_Gunning_FOG_readability,0.986406


In [59]:
alt.Chart(top_corr_feats).mark_rect().encode(
    x=alt.X('Feature_1:O',axis=alt.Axis( 
                                   labelAngle=-45, 
                                   labelOverlap=False,title = "Feature B")),
    y=alt.Y('Feature_2:O',axis=alt.Axis(  
                                   labelOverlap=False,title = "Feature A")),
    color=alt.Color('corr_coef:Q', legend=alt.Legend(title="Absolute Value - Corr Coefficient"))
).properties(title ="Correlation Matrix")

alt.Chart(...)

In [63]:
### Feature Grouping

### Background Sorting

In [127]:
pd.read_csv('C:/Users/NIW/Documents/Capstone Project/data/clean_data/manual_clean_training_dataset_v8.csv').columns

Index(['employee_code', 'rehired_', 'referral_flag', 'job_hopper',
       'competitor_experience', 'Freedom_competitor_exp',
       'Koodo_competitor_exp', 'Shaw_competitor_exp', 'Telus_competitor_exp',
       'Bell_competitor_exp', 'Rogers_competitor_exp',
       'The_Mobile_Shop_competitor_exp', 'Best_Buy_competitor_exp',
       'Videotron_competitor_exp', 'Wow_Mobile_competitor_exp',
       'The_Source_competitor_exp', 'Walmart_competitor_exp',
       'Virgin_Mobile_competitor_exp', 'Osl_competitor_exp',
       'accounting_concentration', 'arts_concentration',
       'business_concentration', 'computer_systems_concentration',
       'engineering_concentration', 'finance_concentration',
       'general_concentration', 'human_resource_concentration',
       'interactive_arts_and_technology_concentration',
       'marketing_concentration', 'not_specified_concentration',
       'other_concentration', 'administrative_jobtitle',
       'assistant_manager_jobtitle', 'blue_collar_jobtitle',

In [128]:
df_1 = pd.read_csv("C:/Users/NIW/Glentel Inc/HR Analytics - Capstone Data/ubc_mds_team_share/make_features/manual_higher_degree.csv")
df_2 = pd.read_csv('C:/Users/NIW/Documents/Capstone Project/data/clean_data/manual_clean_training_dataset_v8.csv')[['employee_code','hp_class','telco_electro_recency','recency_type_telco_electro_exp']]
df_3 = pd.merge(df_1,df_2, on ="employee_code",how = "inner")
df_3['hp_class'] = df_3.hp_class.apply(lambda x: round(x,2)*100)

In [129]:
df_3.columns

Index(['employee_code', 'highest_degree', 'background_highest_degree',
       'country_highest_degree', 'flag_hd_bachelor_plus', 'flag_hd_highschool',
       'business_flag', 'hp_class', 'telco_electro_recency',
       'recency_type_telco_electro_exp'],
      dtype='object')

In [121]:
# regroup background_highest_degree
other_backgrounds = ['audio technician', 'audio technician', 'kinesiology',
                 'blue collar', 'economics', 'sociology',
                 'kinesiology', 'physic', 'statistic', 'hospitality',
                 'criminology', 'english', 'dental', 'human resource',
                 'healthcare', 'communication', 'education']
other_countries = ['dubai', 'pakistan', 'uk', 'china',
                       'england', 'syria', 'indore', 'usa', 'philippine',
                       'vietnam',
                       'brazil', 'turkey', 'united kingdom', 'punjab', 'taiwan',
                       'u']

df_3['background_post_grouping'] = df_3.background_highest_degree.apply(lambda x: "others" if x in other_backgrounds else x)
df_3['country_post_grouping'] = df_3.country_highest_degree.apply(lambda x: "others" if x in other_countries else x)

In [87]:
df_3.columns

Index(['employee_code', 'highest_degree', 'background_highest_degree',
       'country_highest_degree', 'flag_hd_bachelor_plus', 'flag_hd_highschool',
       'business_flag', 'hp_class', 'background_post_grouping'],
      dtype='object')

In [175]:
def plot_var(df,var,nice_name):
    range_color = ["#17becf","green"]
    a = alt.Chart(df).mark_bar().encode(
    #alt.X(var+":O",axis=alt.Axis(title = nice_name)),
    alt.X(var+":O",axis=alt.Axis(labelAngle=0,labelOverlap=False,title = nice_name)),
    #alt.X(var+":O",axis=alt.Axis(title = nice_name,labelAngle=-45,labelOverlap=False),sort=['high','low','inexistant']),
    alt.Y('count()', title='count')
        ).properties(title = "Sample Distribution")

    b = alt.Chart(df).mark_bar().encode(
    #alt.X(var+":O",axis=alt.Axis(labelOverlap=False,title = nice_name)),
    alt.X(var+":O", axis=alt.Axis(labelAngle=0,labelOverlap=False,title = nice_name)),
    #alt.X(var+":O",axis=alt.Axis(title = nice_name,labelAngle=-45,labelOverlap=False),sort=['high','low','inexistant']),
    alt.Y('mean(hp_class):Q', title='High Performer Rate %'),
    color = alt.Color('mean(hp_class)',legend=None,scale=alt.Scale(domain=(0,100), range=range_color) )
        ).properties(title = "High Performance Rate per Value")

    c = alt.vconcat(a, b).resolve_legend(color='independent')
    #c = a|b
    return c
    

In [176]:
chart = plot_var(df_3,"telco_electro_recency","Recency - Job in Electronic/Telco Industry")
#chart.save('C:/Users/NIW/Desktop/images_plots/background_pre.png', scale_factor=2.0)
chart

alt.VConcatChart(...)

**A couple of interesting questions:**
+ Distrition of location and brand - overall and segmented (high and low perf)
+ Average activations per high perf vs average activations of non-high perf
+ Tenure - global and segmented
+ Termination rate - global and segmented

### Distrition of location and brand - overall and segmented (high and low perf)

In [388]:
activations_train_df.columns

Index(['employee_code', 'max_hire_date', 'job_title', 'hp_class', 'tenure',
       'tenure_at_termination', 'termination_type', 'end_perf_window',
       'class_tag', 'hp_perc', 'rehired_', 'referral_flag', 'generation',
       'gender', 'Total', 'Province', 'PostalCode', 'Banner', 'StoreName',
       'RoleName', 'ActType', 'DateCreatedAtMidnight', 'month', 'year',
       'period'],
      dtype='object')

In [179]:
# Load Data
read_train_data_set =pd.read_csv('../../../../data/clean_data/train_dataset.csv',index_col=0,parse_dates=['max_hire_date','termination_date','end_perf_window'],infer_datetime_format=1)
# act_eda_df has all the activations, so we need to join it with the employees what are interested in
activations_train_df = pd.merge(read_train_data_set[['employee_code','max_hire_date','job_title','hp_class','tenure','tenure_at_termination','termination_type']],act_eda_df.drop(columns = ['hp_class','max_hire_date']),on='employee_code',how="inner")

In [422]:
##First just selection activations after latest hiring date
data_a = activations_train_df[activations_train_df.DateCreatedAtMidnight >= activations_train_df.max_hire_date]
## Identifying initial store and location per employee (where they made their first sale)
data_a = data_a[data_a.groupby('employee_code').DateCreatedAtMidnight.transform('min')==data_a['DateCreatedAtMidnight']]
data_a = data_a.drop_duplicates(['employee_code'])
#data_a.head(3)

In [373]:
data_a.head(2)

,employee_code,max_hire_date,job_title,hp_class,tenure,tenure_at_termination,termination_type,end_perf_window,class_tag,hp_perc,...,Province,PostalCode,Banner,StoreName,RoleName,ActType,DateCreatedAtMidnight,month,year,period
31,MZE,2019-05-18,SALES ASSOCIATE,0.0,6,111 days 00:00:00.000000000,Resignation,2020-02-29,non-high-perf,0.0,...,ON,M1P 4P5,WW,WW Scarborough Town Centre Inline 319 1823,XEmployee,COP,2019-05-18,5.0,2019.0,05-2019
135,MRU,2019-02-22,SALES ASSOCIATE,0.0,0,109 days 00:00:00.000000000,Dismissal,2020-02-29,non-high-perf,0.0,...,ON,L7S 2J8,TB,TB Mapleview Shopping Centre Kiosk 429 1177,XEmployee,NAC,2019-02-25,2.0,2019.0,02-2019


In [423]:
### Location
other_provinces = ['NS','MB','SK','QC','NL']
data_a.Province = data_a.Province.apply(lambda x: "others" if x in other_provinces else x)
data_a = data_a.groupby(['Province']).agg(employee_count =('employee_code','count'),high_performers = ('hp_class','mean'))
#data_a = data_a.groupby(['Banner']).agg(employee_count =('employee_code','count'),high_performers = ('hp_class','mean'))
data_a.high_performers = round(data_a.high_performers,2)
data_a['perc'] = round(data_a.employee_count/data_a.employee_count.sum(),2)
data_a['label_pos'] = data_a.employee_count/2
data_a = data_a.reset_index()
data_a.head(2)


,Province,employee_count,high_performers,perc,label_pos
0,AB,43,0.21,0.15,21.5
1,BC,48,0.23,0.17,24.0


In [424]:
def plot_var(df,x_var,y_var_1,y_var_2,nice_name_x_var):
    range_color = ["#17becf","green"]
    a = alt.Chart(df).mark_bar().encode(
    #alt.X(var+":O",axis=alt.Axis(title = nice_name)),
    alt.X(x_var+":O",axis=alt.Axis(labelAngle=-45,labelOverlap=False,title = nice_name_x_var),sort=['ON','BC','AB','NB','others']),
    #alt.X(x_var+":O",axis=alt.Axis(labelAngle=-45,labelOverlap=False,title = nice_name_x_var),sort=['WW','WE','TB']),    
    #alt.X(var+":O",axis=alt.Axis(title = nice_name,labelAngle=-45,labelOverlap=False),sort=['high','low','inexistant']),
    alt.Y(y_var_1, title='count')
        ).properties(title = "Sample Distribution",width=150)
    text_a = a.mark_text(
        align='center',
        baseline='middle',
        #dy=-10  # Nudges text to right so it doesn't appear on top of the bar
        ).encode(
        y = 'label_pos',
        text= alt.Text('perc',format = '.0%')
        )
    b = alt.Chart(df).mark_bar().encode(
    #alt.X(var+":O",axis=alt.Axis(labelOverlap=False,title = nice_name)),
    alt.X(x_var+":O", axis=alt.Axis(labelAngle=-45,labelOverlap=False,title = nice_name_x_var),sort=['ON','BC','AB','NB','others']),
    #alt.X(var+":O",axis=alt.Axis(title = nice_name,labelAngle=-45,labelOverlap=False),sort=['high','low','inexistant']),
    alt.Y(y_var_2+":Q", title='High Performer Rate %',axis=alt.Axis(format='%')),
    color = alt.Color(y_var_2,legend=None,scale=alt.Scale(domain=(0,1), range=range_color) )
        ).properties(title = "High Performance Rate",width=150)
    
    text_b = a.mark_text(
    align='center',
    baseline='middle',
    #dy=-10  # Nudges text to right so it doesn't appear on top of the bar
    ).encode(
    y = y_var_2,
    text= alt.Text(y_var_2,format = '.0%')
    )

    c = alt.hconcat(a+text_a, b+text_b).resolve_legend(color='independent')
    #c = a|b
    return c
    

In [425]:
plot_var(data_a,"Province",'employee_count','high_performers','Province')

alt.HConcatChart(...)

## Average activations per high perf vs average activations of non-high perf

In [387]:
data_b.columns

Index(['employee_code', 'max_hire_date', 'job_title', 'hp_class', 'tenure',
       'tenure_at_termination', 'termination_type', 'end_perf_window',
       'class_tag', 'hp_perc', 'rehired_', 'referral_flag', 'generation',
       'gender', 'Total', 'Province', 'PostalCode', 'Banner', 'StoreName',
       'RoleName', 'ActType', 'DateCreatedAtMidnight', 'month', 'year',
       'period'],
      dtype='object')

In [400]:
## Just select activations after hiring date
data_b = activations_train_df[activations_train_df.DateCreatedAtMidnight >= activations_train_df.max_hire_date]
data_b_1 = data_b.groupby(['month','class_tag','employee_code']).agg(monthly_activations =('Total','sum'))
data_b_1 = data_b_1.reset_index()
data_b_1 = data_b_1.groupby(['class_tag']).agg(average_monthly_activations = ('monthly_activations','mean'))
data_b_1 = data_b_1.reset_index()
data_b_1.average_monthly_activations = round(data_b_1.average_monthly_activations,0)
data_b_1.head(5)

,class_tag,average_monthly_activations
0,high-perf,80.0
1,non-high-perf,41.0


In [414]:
a = alt.Chart(data_b_1).mark_bar().encode(
    #alt.X(var+":O",axis=alt.Axis(title = nice_name)),
    #alt.X(x_var+":O",axis=alt.Axis(labelAngle=-45,labelOverlap=False,title = nice_name_x_var),sort=['ON','BC','AB','NB','others']),
    alt.X("class_tag:O",axis=alt.Axis(labelAngle=-45,labelOverlap=False,title = "Performance Label")),    
    #alt.X(var+":O",axis=alt.Axis(title = nice_name,labelAngle=-45,labelOverlap=False),sort=['high','low','inexistant']),
    alt.Y("average_monthly_activations:Q",scale=alt.Scale(domain=[0, 100]), title='Average Monthly Activations')
        ).properties(title = "High Performers vs Non High Perfomers (Monthly Activations)",width=100)
text_a = a.mark_text(
    align='center',
    baseline='middle',
    dy=-10  # Nudges text to right so it doesn't appear on top of the bar
    ).encode(
    y = 'average_monthly_activations',
    text= alt.Text('average_monthly_activations')
    )

a+text_a

alt.LayerChart(...)

In [447]:
## Just select activations after hiring date
data_b = activations_train_df[activations_train_df.DateCreatedAtMidnight >= activations_train_df.max_hire_date]
other_provinces = ['NS','MB','SK','QC','NL']
data_b.Province = data_b.Province.apply(lambda x: "others" if x in other_provinces else x)
data_b_2 = data_b.groupby(['month','class_tag','employee_code','Province']).agg(monthly_activations =('Total','sum'))
data_b_2 = data_b_2.reset_index()
data_b_2 = data_b_2.groupby(['Province','class_tag']).agg(average_monthly_activations = ('monthly_activations','mean'))
data_b_2 = data_b_2.reset_index()
data_b_2.average_monthly_activations = round(data_b_2.average_monthly_activations,0)
data_b_2.head(10)

,Province,class_tag,average_monthly_activations
0,AB,high-perf,76.0
1,AB,non-high-perf,40.0
2,BC,high-perf,78.0
3,BC,non-high-perf,39.0
4,NB,high-perf,92.0
5,NB,non-high-perf,34.0
6,ON,high-perf,80.0
7,ON,non-high-perf,42.0
8,others,high-perf,82.0
9,others,non-high-perf,42.0


In [449]:
a = alt.Chart(data_b_2).mark_bar().encode(
    #alt.X(var+":O",axis=alt.Axis(title = nice_name)),
    #alt.X(x_var+":O",axis=alt.Axis(labelAngle=-45,labelOverlap=False,title = nice_name_x_var),sort=['ON','BC','AB','NB','others']),
    alt.X("class_tag:O",axis=alt.Axis(labelAngle=-45,labelOverlap=False),title=None),    
    #alt.X(var+":O",axis=alt.Axis(title = nice_name,labelAngle=-45,labelOverlap=False),sort=['high','low','inexistant']),
    alt.Y("average_monthly_activations:Q",scale=alt.Scale(domain=[0, 100]), title='Average Monthly Activations'),
    color = alt.Color('class_tag:N',title = "Performance Label")
        ).properties(width=100)
text_a = a.mark_text(
    align='center',
    baseline='middle',
    dy=-10  # Nudges text to right so it doesn't appear on top of the bar
    ).encode(
    text= alt.Text('average_monthly_activations')
    )

alt.layer(a, text_a, data=data_b_2).facet(column='Province:N')

alt.FacetChart(...)

In [454]:
## Just select activations after hiring date
data_b = activations_train_df[activations_train_df.DateCreatedAtMidnight >= activations_train_df.max_hire_date]
other_provinces = ['NS','MB','SK','QC','NL']
data_b.Province = data_b.Province.apply(lambda x: "others" if x in other_provinces else x)
data_b_3 = data_b.groupby(['month','class_tag','employee_code','Banner']).agg(monthly_activations =('Total','sum'))
data_b_3 = data_b_3.reset_index()
data_b_3 = data_b_3.groupby(['Banner','class_tag']).agg(average_monthly_activations = ('monthly_activations','mean'))
data_b_3 = data_b_3.reset_index()
data_b_3.average_monthly_activations = round(data_b_3.average_monthly_activations,0)
data_b_3.head(10)

,Banner,class_tag,average_monthly_activations
0,TB,high-perf,34.0
1,TB,non-high-perf,29.0
2,WE,high-perf,87.0
3,WE,non-high-perf,59.0
4,WW,high-perf,68.0
5,WW,non-high-perf,32.0


In [456]:
a = alt.Chart(data_b_3).mark_bar().encode(
    #alt.X(var+":O",axis=alt.Axis(title = nice_name)),
    #alt.X(x_var+":O",axis=alt.Axis(labelAngle=-45,labelOverlap=False,title = nice_name_x_var),sort=['ON','BC','AB','NB','others']),
    alt.X("class_tag:O",axis=alt.Axis(labelAngle=-45,labelOverlap=False),title=None),    
    #alt.X(var+":O",axis=alt.Axis(title = nice_name,labelAngle=-45,labelOverlap=False),sort=['high','low','inexistant']),
    alt.Y("average_monthly_activations:Q",scale=alt.Scale(domain=[0, 100]), title='Average Monthly Activations'),
    color = alt.Color('class_tag:N',title = "Performance Label")
        ).properties(width=100)
text_a = a.mark_text(
    align='center',
    baseline='middle',
    dy=-10  # Nudges text to right so it doesn't appear on top of the bar
    ).encode(
    text= alt.Text('average_monthly_activations')
    )

alt.layer(a, text_a, data=data_b_3).facet(column='Banner:N')

alt.FacetChart(...)

### Tenure - global and segmented

In [458]:
read_train_data_set.columns

Index(['employee_code', 'original_hire_date', 'max_hire_date', 'gender',
       'job_title', 'position_status', 'worker_category', 'birth_year',
       'report_date_week_ending_', 'generation', 'tenure', 'tenure_group',
       'qty', 'termination_date', 'termination_reason', 'termination_type',
       'tenure_at_termination', 'end_perf_window', 'tenure_at_end_perf_window',
       'employee_name', 'language', 'resume_found', 'months_high_perf',
       'months_with_perf', 'hp_perc', 'hp_class', 'perf_found', 'rehired_',
       'referral_flag', 'exclusion_code'],
      dtype='object')

In [511]:
data_c = read_train_data_set[['employee_code','max_hire_date','termination_date','hp_class','tenure','tenure_at_termination','termination_reason','termination_type','tenure_at_end_perf_window','class_tag']]

KeyError: "['class_tag'] not in index"

In [512]:
data_c.tenure_at_termination = pd.to_timedelta(data_c.tenure_at_termination,'days')
data_c.tenure_at_end_perf_window = pd.to_timedelta(data_c.tenure_at_end_perf_window,'days')

In [503]:
### Termination helpers
def termination_general(df):
    if df['tenure_at_termination'] >= timedelta(days=0):
        return 1
    else:
        return 0
    
def final_tenure(df):
    if df['termination_flag'] ==1:
        return df['tenure_at_termination'].days
    else:
        return df['tenure_at_end_perf_window'].days

In [505]:
data_c['termination_flag'] = data_c.apply(lambda x: termination_general(x),axis = 1)
data_c['final_tenure_days'] = data_c.apply(lambda x: final_tenure(x),axis=1)

C:\Users\NIW\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\NIW\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [507]:
data_c.head(5)

,employee_code,max_hire_date,termination_date,hp_class,tenure,tenure_at_termination,termination_reason,termination_type,tenure_at_end_perf_window,termination_flag,final_tenure_days
2825,MZE,2019-05-18,2019-09-06,0.0,6,111 days,Quit - Return to School,Resignation,287 days,1,111
2639,MRU,2019-02-22,2019-06-11,0.0,0,109 days,Dismissal,Dismissal,372 days,1,109
1361,LAP,2019-04-25,2018-06-16,0.0,1,-313 days,Quit - Personal,Resignation,310 days,0,310
2799,MYD,2019-06-03,NaT,1.0,4,NaT,NaN,NaN,271 days,0,271
668,JJB,2019-10-04,2018-09-30,0.0,0,-369 days,Quit - Return to School,Resignation,148 days,0,148


In [508]:
data_c.columns

Index(['employee_code', 'max_hire_date', 'termination_date', 'hp_class',
       'tenure', 'tenure_at_termination', 'termination_reason',
       'termination_type', 'tenure_at_end_perf_window', 'termination_flag',
       'final_tenure_days'],
      dtype='object')

In [510]:
data_c.final_tenure_days.mean()

227.97916666666666

In [514]:
data_c['class_tag'] =data_c.hp_class.apply(lambda x: "high-perf" if x ==1 else "non-high-perf")

C:\Users\NIW\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [534]:
data_c_1 = data_c.groupby(['class_tag']).agg(tenure_days = ('final_tenure_days','mean'),
                                            termination_rate =('termination_flag','mean')).reset_index().round(2)
data_c_1.tenure_days = data_c_1.tenure_days.astype(int)
data_c_1['label_pos'] = data_c_1.tenure_days/2
data_c_1

,class_tag,tenure_days,termination_rate,label_pos
0,high-perf,275,0.07,137.5
1,non-high-perf,212,0.36,106.0


In [550]:
def plot_var(df,x_var,y_var_1,y_var_2,nice_name_x_var):
    range_color = ["#17becf","green"]
    a = alt.Chart(df).mark_bar().encode(
    #alt.X(var+":O",axis=alt.Axis(title = nice_name)),
    alt.X(x_var+":O",axis=alt.Axis(labelAngle=-45,labelOverlap=False,title = nice_name_x_var),sort=['ON','BC','AB','NB','others']),
    #alt.X(x_var+":O",axis=alt.Axis(labelAngle=-45,labelOverlap=False,title = nice_name_x_var),sort=['WW','WE','TB']),    
    #alt.X(var+":O",axis=alt.Axis(title = nice_name,labelAngle=-45,labelOverlap=False),sort=['high','low','inexistant']),
    alt.Y(y_var_1, title='Tenure(days)', scale =alt.Scale(domain=[0,300]))
        ).properties(title = "Tenure Comparison",width=150)
    text_a = a.mark_text(
        align='center',
        baseline='middle',
        dy=-10  # Nudges text to right so it doesn't appear on top of the bar
        ).encode(
        #y = 'label_pos',
        text= alt.Text('tenure_days')
        )
    b = alt.Chart(df).mark_bar().encode(
    #alt.X(var+":O",axis=alt.Axis(labelOverlap=False,title = nice_name)),
    alt.X(x_var+":O", axis=alt.Axis(labelAngle=-45,labelOverlap=False,title = nice_name_x_var),sort=['ON','BC','AB','NB','others']),
    #alt.X(var+":O",axis=alt.Axis(title = nice_name,labelAngle=-45,labelOverlap=False),sort=['high','low','inexistant']),
    alt.Y(y_var_2+":Q", title='Termination Rate %',axis=alt.Axis(format='%')),
    color = alt.Color(y_var_2,legend=None,scale=alt.Scale(domain=(0,1), range=range_color) )
        ).properties(title = "Termination Rate",width=150)
    
    text_b = a.mark_text(
    align='center',
    baseline='middle',
    dy=-10  # Nudges text to right so it doesn't appear on top of the bar
    ).encode(
    y = y_var_2,
    text= alt.Text(y_var_2,format = '.0%')
    )

    c = alt.hconcat(a+text_a, b+text_b).resolve_legend(color='independent')
    #c = a|b
    return c
    

In [551]:
plot_var(data_c_1,"class_tag","tenure_days","termination_rate","Performance Label")

alt.HConcatChart(...)

In [567]:
terminated_non_high_perf = data_c[(data_c['class_tag']=="non-high-perf") & (data_c['termination_flag']==1)]
terminated_high_perf = data_c[(data_c['class_tag']=="high-perf") & (data_c['termination_flag']==1)]

In [570]:
t_nh_1 = terminated_non_high_perf.groupby(['termination_type']).agg(employee_count = ('employee_code','count'))
t_nh_1['perc'] = round(t_nh_1.employee_count/t_nh_1.employee_count.sum(),2)
t_nh_1

,employee_count,perc
termination_type,,
Dismissal,22,0.28
Resignation,56,0.72


In [574]:
not_others = ['Quit - Personal','Quit - New Job - Better Opportunity','Quit - Return to School','Quit - Job Abandonment - Attendance']

In [577]:
t_nh_2 = terminated_non_high_perf[terminated_non_high_perf['termination_type']=='Resignation']
t_nh_2.termination_reason = t_nh_2.termination_reason.apply(lambda x: x if x in not_others else "other")
t_nh_2 = t_nh_2.groupby(['termination_reason']).agg(employee_count = ('employee_code','count')).reset_index()
t_nh_2['perc'] = round(t_nh_2.employee_count/t_nh_2.employee_count.sum(),2)
t_nh_2

,termination_reason,employee_count,perc
0,Quit - Job Abandonment - Attendance,3,0.05
1,Quit - New Job - Better Opportunity,14,0.25
2,Quit - Personal,23,0.41
3,Quit - Return to School,7,0.12
4,other,9,0.16


In [587]:
a = alt.Chart(t_nh_2).mark_bar().encode(
    #alt.X(var+":O",axis=alt.Axis(title = nice_name)),
    #alt.X(x_var+":O",axis=alt.Axis(labelAngle=-45,labelOverlap=False,title = nice_name_x_var),sort=['ON','BC','AB','NB','others']),
    alt.X("termination_reason:O",axis=alt.Axis(labelAngle=-45,labelOverlap=False,title = "Termination Reason"),sort=['Quit - Personal','Quit - New Job - Better Opportunity','Quit - Return to School','Quit - Job Abandonment - Attendance','other']),    
    #alt.X(var+":O",axis=alt.Axis(title = nice_name,labelAngle=-45,labelOverlap=False),sort=['high','low','inexistant']),
    alt.Y("employee_count:Q",scale=alt.Scale(domain=[0, 30]), title='Employee Counts')
        ).properties(title = "Resignation Reason - Non-high Performers ",width=200)
text_a = a.mark_text(
    align='center',
    baseline='middle',
    dy=-10  # Nudges text to right so it doesn't appear on top of the bar
    ).encode(
    #y = 'average_monthly_activations',
    text= alt.Text('perc',format = "0.0%")
    )

a+text_a

alt.LayerChart(...)

In [568]:
t_h_1 = terminated_high_perf.groupby(['termination_reason']).agg(employee_count = ('employee_code','count'))
t_h_1['perc'] = round(t_h_1.employee_count/t_h_1.employee_count.sum(),2)
t_h_1

,employee_count,perc
termination_reason,,
Quit - Moved - Out of Country,1,0.2
Quit - New Job - Better Opportunity,1,0.2
Quit - Personal,3,0.6
